In [92]:
from pathlib import Path

# single source of truth for dashboard outputs
OUTPUT_DIR = Path("docs/data/derived")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def save_csv(df, name):
    p = OUTPUT_DIR / name
    df.to_csv(p, index=False)
    print(f"✔ saved: {p}")

# …then everywhere you save:
# save_csv(provider_scores_df, "provider_scores_latest.csv")
# save_csv(roi_df,              "roi_comparison.csv")


In [93]:
#scrape provider
# --- Async helper that works in notebooks and GitHub Actions ---
import asyncio

def await_safe(coro):
    """
    Run an async coroutine from anywhere:
    - If an event loop is already running (Jupyter/nbconvert), use nest_asyncio + run_until_complete
    - Otherwise, use asyncio.run
    """
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            try:
                import nest_asyncio
                nest_asyncio.apply()
            except Exception:
                pass
            return loop.run_until_complete(coro)
        else:
            return asyncio.run(coro)
    except RuntimeError:
        # No current loop
        return asyncio.run(coro)


In [94]:
# Lambda Labs 

import re, requests, pandas as pd
from typing import Optional
from bs4 import BeautifulSoup
from datetime import datetime, timezone

HEADERS = {"User-Agent": "Mozilla/5.0"}

SLIM_COLS = [
    "provider","region","gpu_model","type","duration","gpu_count",
    "price_hourly_usd","source_url","fetched_at_utc"
]

def _now_iso() -> str:
    return datetime.now(timezone.utc).isoformat(timespec="seconds")

def _ensure_slim(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in SLIM_COLS:
        if c not in out.columns: out[c] = None
    out["price_hourly_usd"] = pd.to_numeric(out["price_hourly_usd"], errors="coerce")
    out["fetched_at_utc"] = pd.to_datetime(out["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
    return out[SLIM_COLS]

def _norm_gpu(s: str) -> str:
    s = re.sub(r"\bon[-\s]?demand\b", "", s, flags=re.I)
    s = s.replace("NVIDIA", "").strip()
    s = re.sub(r"\s+", " ", s)
    s = s.upper().replace("GH200", "H200")  # treat GH200 as H200
    return s.strip()

def _gpu_count(s: str) -> Optional[int]:
    if not isinstance(s, str): 
        return None
    m = re.search(r"(\d+)x", s, flags=re.I)
    return int(m.group(1)) if m else None

def _price_in(text: str) -> Optional[float]:
    if not isinstance(text, str): 
        return None
    m = re.search(r"\$\s*([0-9]+(?:\.[0-9]+)?)", text.replace(",", ""))
    return float(m.group(1)) if m else None

def _infer_region(table) -> str:
    hdr = table.find_previous(["h2","h3","h4","p"])
    if hdr:
        t = hdr.get_text(" ", strip=True).lower()
        if "europe" in t or "eu" in t: return "EU"
        if "united states" in t or "us" in t or "usa" in t: return "US"
    return "US"

def scrape_lambda_labs(region: Optional[str] = None) -> pd.DataFrame:
    """
    Scrapes https://cloud.lambdalabs.com/pricing and returns SLIM rows
    for H100/H200 (On-Demand, 1h). If `region` provided, overrides detected region.
    """
    url = "https://cloud.lambdalabs.com/pricing"
    r = requests.get(url, headers=HEADERS, timeout=30)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "html.parser")

    rows_out = []
    tables = soup.find_all("table")
    for table in tables:
        tbl_region = region or _infer_region(table)
        for tr in table.find_all("tr"):
            tds = [td.get_text(" ", strip=True) for td in tr.find_all("td")]
            if not tds:
                continue
            row_text = " | ".join(tds)

            if not (re.search(r"\bH100\b", row_text, re.I) or re.search(r"\bH200\b|\bGH200\b", row_text, re.I)):
                continue

            price = _price_in(row_text)
            if price is None:
                continue

            
            gpu_cell = next((c for c in tds if ("H100" in c.upper() or "H200" in c.upper() or "GH200" in c.upper())), None)
            gpu_model = _norm_gpu(gpu_cell or ("H100" if "H100" in row_text.upper() else "H200"))
            count = _gpu_count(gpu_model)

            rows_out.append({
                "provider": "Lambda Labs",
                "region": tbl_region,
                "gpu_model": gpu_model,      
                "type": "On-Demand",
                "duration": "1h",
                "gpu_count": count,
                "price_hourly_usd": price,
                "source_url": url,
                "fetched_at_utc": _now_iso(),
            })

    df = pd.DataFrame(rows_out)
    if df.empty:
        return _ensure_slim(df)
    keep = df["gpu_model"].str.contains(r"\bH100\b|\bH200\b", regex=True, na=False)
    df = df[keep].reset_index(drop=True)
    return _ensure_slim(df)

# Example:
df_lambda = scrape_lambda_labs(region="US")
display(df_lambda.head())



,provider,region,gpu_model,type,duration,gpu_count,price_hourly_usd,source_url,fetched_at_utc
0,Lambda Labs,US,8X H100 SXM,On-Demand,1h,8,2.99,https://cloud.lambdalabs.com/pricing,2025-09-10 12:21:30
1,Lambda Labs,US,4X H100 SXM,On-Demand,1h,4,3.09,https://cloud.lambdalabs.com/pricing,2025-09-10 12:21:30
2,Lambda Labs,US,2X H100 SXM,On-Demand,1h,2,3.19,https://cloud.lambdalabs.com/pricing,2025-09-10 12:21:30
3,Lambda Labs,US,1X H200,On-Demand,1h,1,1.49,https://cloud.lambdalabs.com/pricing,2025-09-10 12:21:30
4,Lambda Labs,US,1X H100 SXM,On-Demand,1h,1,3.29,https://cloud.lambdalabs.com/pricing,2025-09-10 12:21:30


In [95]:
# ================= SFCompute — H100/H200 scraper (slim schema) =================
# Output files:
#   docs/data/history/sfcompute_history.csv
#   docs/data/latest/sfcompute_latest.csv
#
# Notes:
# - Tries to capture *per GPU-hour* prices directly (e.g. "per GPU hour", "per card-hour").
# - If only instance price is shown with "x GPUs", we divide by the parsed GPU count.
# - Very defensive: regex-only parsing; ignores out-of-band prices.
# - Region defaults to "Global" (adjust mapping below if their page shows regions).
#
# Requires: pandas, playwright (already in your CI), requests, bs4 (optional)

import re, asyncio, pandas as pd, tempfile, requests
from bs4 import BeautifulSoup
from datetime import datetime, timezone
from pathlib import Path
from playwright.async_api import async_playwright

# --------------------- storage + schema helpers ---------------------
SLIM_COLS = [
    "provider","region","gpu_model","type","duration","gpu_count",
    "price_hourly_usd","source_url","fetched_at_utc"
]
BASE = Path("docs/data")
HIST_DIR = BASE / "history"
LATEST_DIR = BASE / "latest"
for d in (HIST_DIR, LATEST_DIR):
    d.mkdir(parents=True, exist_ok=True)

def _now_iso() -> str:
    return datetime.now(timezone.utc).isoformat(timespec="seconds")

def _ensure_slim(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in SLIM_COLS:
        if c not in out.columns: out[c] = None
    out["price_hourly_usd"] = pd.to_numeric(out["price_hourly_usd"], errors="coerce")
    out["fetched_at_utc"] = pd.to_datetime(out["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
    return out[SLIM_COLS]

def _safe_to_csv(df: pd.DataFrame, path: Path):
    try:
        path.parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(path, index=False)
        return path
    except Exception:
        tmp = Path(tempfile.gettempdir()) / path.name
        df.to_csv(tmp, index=False)
        return tmp

def _save_provider(df: pd.DataFrame, provider_slug: str):
    df = _ensure_slim(df)
    # history append + dedupe
    hist_path = HIST_DIR / f"{provider_slug}_history.csv"
    if hist_path.exists():
        old = pd.read_csv(hist_path, low_memory=False)
        old["fetched_at_utc"] = pd.to_datetime(old["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
        all_df = pd.concat([old, df], ignore_index=True)
    else:
        all_df = df.copy()
    all_df = (
        all_df.dropna(subset=["gpu_model","price_hourly_usd"])
              .drop_duplicates(subset=["provider","region","gpu_model","type","duration","price_hourly_usd","fetched_at_utc"], keep="last")
              .sort_values("fetched_at_utc")
    )
    hist_out = _safe_to_csv(all_df, hist_path)

    # latest (newest per gpu/type/region/duration)
    key = ["gpu_model","type","region","duration"]
    latest = all_df.sort_values("fetched_at_utc").drop_duplicates(subset=key, keep="last")
    latest_out = _safe_to_csv(latest, LATEST_DIR / f"{provider_slug}_latest.csv")

    print(f"[sfcompute] history  -> {hist_out}")
    print(f"[sfcompute] latest   -> {latest_out}")
    return latest

# --------------------- scraping patterns ---------------------
GPU_RE = r"(H100|H200)\b"
USD_HOURLY = re.compile(r"\$\s*([0-9]+(?:\.[0-9]+)?)\s*(?:/|\s*(?:per|an)\s*)?(?:h|hr|hour)\b", re.I)
USD_PER_GPU_HR = re.compile(r"\$\s*([0-9]+(?:\.[0-9]+)?)\s*(?:/|\s*(?:per|an)\s*)?(?:gpu|card)[-\s]?(?:h|hr|hour)\b", re.I)
COUNT_PATS = [
    re.compile(r"(\d+)\s*[×x]\s*(?:GPUs?|cards?)\b", re.I), # "8x GPUs"
    re.compile(r"(?:GPUs?|cards?)\s*[:\-]?\s*(\d+)\b", re.I), # "GPUs: 8"
    re.compile(r"\b(\d+)\s*[×x]\b", re.I),                  # "8x"
]
MODEL_NEAR_PRICE = re.compile(rf"{GPU_RE}.*?\$\s*([0-9]+(?:\.[0-9]+)?)", re.I | re.S)

def _parse_count(text: str):
    for pat in COUNT_PATS:
        m = pat.search(text)
        if m:
            try:
                n = int(m.group(1))
                if 1 <= n <= 64:
                    return n
            except: 
                pass
    return None

def _detect_region(text: str) -> str:
    t = text.lower()
    if "us" in t or "usa" in t or "america" in t: return "US"
    if "europe" in t or "eu " in t or " eu-" in t: return "EU"
    if "asia" in t or "apac" in t: return "APAC"
    return "Global"

async def _fetch_text_playwright(url: str, wait: str = "domcontentloaded") -> str:
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, timeout=90000, wait_until=wait)
        # light scroll to trigger lazy content
        for _ in range(2):
            await page.evaluate("window.scrollBy(0, document.body.scrollHeight)")
            await page.wait_for_timeout(600)
        body = await page.content()
        await browser.close()
    # get text from HTML
    soup = BeautifulSoup(body, "html.parser")
    return soup.get_text(" ", strip=True)

def _fetch_text_requests(url: str) -> str:
    html = requests.get(url, headers={"User-Agent":"Mozilla/5.0"}, timeout=60).text
    soup = BeautifulSoup(html, "html.parser")
    return soup.get_text(" ", strip=True)

# --------------------- main scrape ---------------------
async def scrape_sfcompute(urls=None) -> pd.DataFrame:
    """
    Tries a list of candidate pages and extracts H100/H200 prices.
    Adjust `urls` if SFCompute hosts pricing elsewhere.
    """
    if urls is None:
        urls = [
            "https://sfcompute.com",              # root
            "https://sfcompute.com/pricing",      # common pattern
            "https://www.sfcompute.com/pricing",
            "https://www.sfcompute.com"           # fallback
        ]

    rows = []
    fetched_any = False

    for url in urls:
        try:
            # try Playwright first (handles dynamic sites), fallback to requests
            try:
                text = await _fetch_text_playwright(url)
            except Exception:
                text = _fetch_text_requests(url)

            fetched_any = True
            norm = re.sub(r"\s+", " ", text)
            up = norm.upper()

            # quick region hint from the page (very rough)
            region = _detect_region(norm)

            # 1) Direct per-GPU/hour prices
            for m in re.finditer(rf"{GPU_RE}.*?{USD_PER_GPU_HR.pattern}", norm, flags=re.I):
                seg = m.group(0)
                gm = re.search(GPU_RE, seg, flags=re.I)
                pm = re.search(USD_PER_GPU_HR, seg, flags=re.I)
                if not (gm and pm): 
                    continue
                model = gm.group(1).upper()
                price = float(pm.group(1))
                if not (0.2 <= price <= 50.0): 
                    continue
                rows.append({
                    "provider": "SFCompute",
                    "region": region,
                    "gpu_model": model,
                    "type": "On-Demand",
                    "duration": "1h",
                    "gpu_count": 1,
                    "price_hourly_usd": price,
                    "source_url": url,
                    "fetched_at_utc": _now_iso(),
                })

            # 2) Instance price + GPU count nearby → per-GPU
            #    Look for "... H100 ... $X /hour ... GPUs: N ..."
            for m in re.finditer(MODEL_NEAR_PRICE, norm):
                model = m.group(1).upper()
                instance_price = float(m.group(2))
                # sanity on instance price band
                if not (0.5 <= instance_price <= 1000):
                    continue
                # search a local window around the match for counts
                start = max(0, m.start() - 120)
                end   = min(len(norm), m.end() + 120)
                window = norm[start:end]
                cnt = _parse_count(window) or 1
                per_gpu = instance_price / max(1, cnt)
                if not (0.2 <= per_gpu <= 50.0): 
                    continue
                rows.append({
                    "provider": "SFCompute",
                    "region": region,
                    "gpu_model": model,
                    "type": "On-Demand",
                    "duration": "1h",
                    "gpu_count": int(cnt),
                    "price_hourly_usd": round(per_gpu, 4),
                    "source_url": url,
                    "fetched_at_utc": _now_iso(),
                })

        except Exception as e:
            print(f"[WARN] sfcompute fetch failed for {url}: {e}")

    df = pd.DataFrame(rows, columns=SLIM_COLS)
    if df.empty and fetched_any:
        # create empty slim to keep pipeline happy
        df = _ensure_slim(pd.DataFrame([], columns=SLIM_COLS))
    elif not df.empty:
        # de-dupe by (model, price)
        df = (df.sort_values(["gpu_model","price_hourly_usd","fetched_at_utc"])
                .drop_duplicates(subset=["gpu_model","price_hourly_usd"], keep="last")
                .reset_index(drop=True))
    return df

# --------------------- run & save ---------------------
def arun(coro):
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:
        return asyncio.run(coro)
    else:
        import nest_asyncio; nest_asyncio.apply()
        return loop.run_until_complete(coro)

df_sf = arun(scrape_sfcompute())
latest_sf = _save_provider(df_sf, "sfcompute")
display(df_sf.head(20))


[sfcompute] history  -> docs/data/history/sfcompute_history.csv
[sfcompute] latest   -> docs/data/latest/sfcompute_latest.csv


,provider,region,gpu_model,type,duration,gpu_count,price_hourly_usd,source_url,fetched_at_utc
0,SFCompute,US,H200,On-Demand,1h,1,1.38,https://www.sfcompute.com,2025-09-10T12:21:51+00:00
1,SFCompute,US,H200,On-Demand,1h,1,1.40,https://www.sfcompute.com,2025-09-10T12:21:51+00:00


In [96]:
# ===== Lambda Labs (static) + RunPod (async) with per-provider history =====
# Slim schema: provider, region, gpu_model, type, duration, gpu_count,
#              price_hourly_usd, source_url, fetched_at_utc
# Py 3.8 compatible

import re, os, asyncio, pandas as pd, tempfile
from typing import Optional, Dict, Any
from bs4 import BeautifulSoup
from datetime import datetime, timezone
from pathlib import Path
import requests

HEADERS = {"User-Agent": "Mozilla/5.0"}
SLIM_COLS = [
    "provider","region","gpu_model","type","duration","gpu_count",
    "price_hourly_usd","source_url","fetched_at_utc"
]

# -------- storage (per-provider history/snapshots) --------
BASE = Path("docs/data")
HIST_DIR = BASE / "history"
SNAP_DIR = BASE / "snapshots"
LATEST_DIR = BASE / "latest"
for d in (HIST_DIR, SNAP_DIR, LATEST_DIR):
    try:
        d.mkdir(parents=True, exist_ok=True)
    except Exception:
        # fallback to tmp if workspace is read-only
        tmp = Path(tempfile.gettempdir()) / "gpu_data"
        d = tmp / d.name
        d.mkdir(parents=True, exist_ok=True)

def _now_iso() -> str:
    return datetime.now(timezone.utc).isoformat(timespec="seconds")

def _ensure_slim(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in SLIM_COLS:
        if c not in out.columns: out[c] = None
    out["price_hourly_usd"] = pd.to_numeric(out["price_hourly_usd"], errors="coerce")
    out["fetched_at_utc"] = pd.to_datetime(out["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
    return out[SLIM_COLS]

def _save_provider(df: pd.DataFrame, provider_slug: str):
    df = _ensure_slim(df)
    ts = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%S")
    # snapshot
    snap_path = SNAP_DIR / f"{ts}_{provider_slug}.csv"
    try:
        df.to_csv(snap_path, index=False)
    except Exception:
        snap_path = Path(tempfile.gettempdir()) / f"{ts}_{provider_slug}.csv"
        df.to_csv(snap_path, index=False)
    # history (append + dedupe)
    hist_path = HIST_DIR / f"{provider_slug}_history.csv"
    if hist_path.exists():
        old = pd.read_csv(hist_path, low_memory=False)
        old["fetched_at_utc"] = pd.to_datetime(old["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
        all_df = pd.concat([old, df], ignore_index=True)
    else:
        all_df = df.copy()
    all_df = (all_df
              .dropna(subset=["gpu_model","price_hourly_usd"])
              .drop_duplicates(subset=["provider","region","gpu_model","type","duration",
                                       "fetched_at_utc","price_hourly_usd"], keep="last")
              .sort_values("fetched_at_utc"))
    try:
        all_df.to_csv(hist_path, index=False)
    except Exception:
        hist_path = Path(tempfile.gettempdir()) / f"{provider_slug}_history.csv"
        all_df.to_csv(hist_path, index=False)
    # latest (newest rows only per gpu/type/region/duration)
    key = ["gpu_model","type","region","duration"]
    latest = all_df.sort_values("fetched_at_utc").drop_duplicates(subset=key, keep="last")
    latest_path = LATEST_DIR / f"{provider_slug}_latest.csv"
    try:
        latest.to_csv(latest_path, index=False)
    except Exception:
        latest_path = Path(tempfile.gettempdir()) / f"{provider_slug}_latest.csv"
        latest.to_csv(latest_path, index=False)
    print(f"[{provider_slug}] snapshot -> {snap_path}\n[{provider_slug}] history  -> {hist_path}\n[{provider_slug}] latest   -> {latest_path}")
    return latest

# ------------------------- Lambda Labs (static) -------------------------
def _norm_gpu_lambda(s: str) -> str:
    s = re.sub(r"\bon[-\s]?demand\b", "", s, flags=re.I)
    s = s.replace("NVIDIA", "").strip()
    s = re.sub(r"\s+", " ", s)
    s = s.upper().replace("GH200", "H200")
    return s.strip()

def _gpu_count(text: str) -> Optional[int]:
    if not isinstance(text, str): return None
    m = re.search(r"(\d+)\s*x", text, flags=re.I)
    return int(m.group(1)) if m else None

def _price_dollar(text: str) -> Optional[float]:
    if not isinstance(text, str): return None
    m = re.search(r"\$\s*([0-9]+(?:\.[0-9]+)?)", text.replace(",", ""))
    return float(m.group(1)) if m else None

def _infer_region_lambda(table) -> str:
    hdr = table.find_previous(["h2","h3","h4","p"])
    if hdr:
        t = hdr.get_text(" ", strip=True).lower()
        if "europe" in t or "eu" in t: return "EU"
        if "united states" in t or "us" in t or "usa" in t: return "US"
    return "US"

def scrape_lambda_labs(region: Optional[str] = None) -> pd.DataFrame:
    url = "https://cloud.lambdalabs.com/pricing"
    r = requests.get(url, headers=HEADERS, timeout=30); r.raise_for_status()
    soup = BeautifulSoup(r.text, "html.parser")

    out = []
    for table in soup.find_all("table"):
        tbl_region = region or _infer_region_lambda(table)
        for tr in table.find_all("tr"):
            tds = [td.get_text(" ", strip=True) for td in tr.find_all("td")]
            if not tds: continue
            row_text = " | ".join(tds)
            if not (re.search(r"\bH100\b", row_text, re.I) or re.search(r"\bH200\b|\bGH200\b", row_text, re.I)):
                continue
            price = _price_dollar(row_text)
            if price is None: continue
            gpu_cell = next((c for c in tds if ("H100" in c.upper() or "H200" in c.upper() or "GH200" in c.upper())), None)
            model = _norm_gpu_lambda(gpu_cell or ("H100" if "H100" in row_text.upper() else "H200"))
            out.append({
                "provider": "Lambda Labs",
                "region": tbl_region,
                "gpu_model": model,
                "type": "On-Demand",
                "duration": "1h",
                "gpu_count": _gpu_count(model),
                "price_hourly_usd": price,
                "source_url": url,
                "fetched_at_utc": _now_iso(),
            })
    df = pd.DataFrame(out)
    if df.empty: return _ensure_slim(df)
    keep = df["gpu_model"].str.contains(r"\bH100\b|\bH200\b", regex=True, na=False)
    return _ensure_slim(df[keep].reset_index(drop=True))

# --------------------------- RunPod (async) ---------------------------
def _extract_gpu_model_runpod(text: str) -> Optional[str]:
    if not isinstance(text, str): return None
    text_up = re.sub(r"\s+", " ", text.upper())
    m = re.search(r"(H(?:100|200)(?:\s*(?:SXM|PCIE|NVL))?(?:\s*\d{2,3}\s*GB)?)", text_up)
    return m.group(1).strip() if m else None

def _price_hourly_runpod(text: str) -> Optional[float]:
    if not isinstance(text, str): return None
    t = text.replace(",", "")
    m = re.search(r"\$\s*([0-9]+(?:\.[0-9]+)?)\s*(?:/|\s*(?:per|an)\s*)?(?:h|hr|hour)\b", t, flags=re.I)
    return float(m.group(1)) if m else None

async def scrape_runpod_async() -> pd.DataFrame:
    from playwright.async_api import async_playwright
    url = "https://www.runpod.io/pricing"; region = "Global"

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, wait_until="domcontentloaded", timeout=60000)
        await page.evaluate("window.scrollTo(0, document.body.scrollHeight)")
        await page.wait_for_timeout(1200)
        html = await page.content()
        await browser.close()

    soup = BeautifulSoup(html, "html.parser")
    nodes = soup.find_all(["section","div","article","li","tr"], class_=re.compile(r"(price|pricing|card|grid|table)", re.I))
    if not nodes:
        nodes = soup.find_all(["section","div","article","li","tr","p","span"])

    out = []
    for n in nodes:
        text = n.get_text(" ", strip=True)
        if "H100" not in text and "H200" not in text: 
            continue
        price = _price_hourly_runpod(text)
        if price is None:
            continue
        model = _extract_gpu_model_runpod(text)
        if model is None:
            continue
        out.append({
            "provider": "RunPod",
            "region": region,
            "gpu_model": model,
            "type": "On-Demand",
            "duration": "1h",
            "gpu_count": _gpu_count(text),
            "price_hourly_usd": price,
            "source_url": url,
            "fetched_at_utc": _now_iso(),
        })
    df = pd.DataFrame(out)
    if df.empty: return _ensure_slim(df)
    df = df[df["gpu_model"].str.contains(r"\bH100\b|\bH200\b", na=False)]
    df = df[(df["price_hourly_usd"] > 0) & (df["price_hourly_usd"] < 200)].reset_index(drop=True)
    return _ensure_slim(df)

# --------------- Runner that works in scripts & notebooks ---------------
def arun(coro):
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:
        return asyncio.run(coro)
    else:
        import nest_asyncio; nest_asyncio.apply()
        return loop.run_until_complete(coro)

# ------------------------------ RUN --------------------------------
# Lambda Labs
df_lambda = scrape_lambda_labs(region="US")
_save_provider(df_lambda, "lambda_labs")

# RunPod
df_runpod = arun(scrape_runpod_async())
_save_provider(df_runpod, "runpod")


[lambda_labs] snapshot -> docs/data/snapshots/20250910_122153_lambda_labs.csv
[lambda_labs] history  -> docs/data/history/lambda_labs_history.csv
[lambda_labs] latest   -> docs/data/latest/lambda_labs_latest.csv
[runpod] snapshot -> docs/data/snapshots/20250910_122156_runpod.csv
[runpod] history  -> docs/data/history/runpod_history.csv
[runpod] latest   -> docs/data/latest/runpod_latest.csv


,provider,region,gpu_model,type,duration,gpu_count,price_hourly_usd,source_url,fetched_at_utc
6,RunPod,Global,H200 141 GB,On-Demand,1h,None,3.59,https://www.runpod.io/pricing,2025-09-10 12:21:56
8,RunPod,Global,H100 PCIE 80 GB,On-Demand,1h,None,1.99,https://www.runpod.io/pricing,2025-09-10 12:21:56


In [97]:
# --- Nebius H100/H200 scraper (uses YOUR parsing + per-provider history) ---

import re, time, tempfile, requests, pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
from pathlib import Path
import pytz

# ---------- your original config ----------
url = "https://nebius.com/prices"
UA = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124 Safari/537.36"}
TZ = pytz.utc
MIN_PRICE, MAX_PRICE = 0.3, 20.0   # sanity for $/GPU/hr

# ---------- storage dirs (snapshot/history/latest) ----------
SLIM_COLS = [
    "provider","region","gpu_model","type","duration","gpu_count",
    "price_hourly_usd","source_url","fetched_at_utc"
]
BASE = Path("docs/data")
HIST_DIR = BASE / "history"
SNAP_DIR = BASE / "snapshots"
LATEST_DIR = BASE / "latest"
for d in (HIST_DIR, SNAP_DIR, LATEST_DIR):
    try:
        d.mkdir(parents=True, exist_ok=True)
    except Exception:
        pass  # we'll fall back to tmp if write fails later

def _ensure_slim(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in SLIM_COLS:
        if c not in out.columns: out[c] = None
    out["price_hourly_usd"] = pd.to_numeric(out["price_hourly_usd"], errors="coerce")
    out["fetched_at_utc"] = pd.to_datetime(out["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
    return out[SLIM_COLS]

def _safe_to_csv(df: pd.DataFrame, path: Path):
    try:
        path.parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(path, index=False)
        return path
    except Exception:
        tmp = Path(tempfile.gettempdir()) / path.name
        df.to_csv(tmp, index=False)
        return tmp

def _save_provider(df: pd.DataFrame, provider_slug: str):
    df = _ensure_slim(df)
    ts = datetime.now(TZ).strftime("%Y%m%d_%H%M%S")

    # snapshot
    snap_path = SNAP_DIR / f"{ts}_{provider_slug}.csv"
    snap_path = _safe_to_csv(df, snap_path)

    # history (append + dedupe)
    hist_path = HIST_DIR / f"{provider_slug}_history.csv"
    if hist_path.exists():
        old = pd.read_csv(hist_path, low_memory=False)
        old["fetched_at_utc"] = pd.to_datetime(old["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
        all_df = pd.concat([old, df], ignore_index=True)
    else:
        all_df = df.copy()
    all_df = (all_df
              .dropna(subset=["gpu_model","price_hourly_usd"])
              .drop_duplicates(subset=["provider","region","gpu_model","type","duration",
                                       "fetched_at_utc","price_hourly_usd"], keep="last")
              .sort_values("fetched_at_utc"))
    hist_path = _safe_to_csv(all_df, hist_path)

    # latest (newest per gpu/type/region/duration)
    key = ["gpu_model","type","region","duration"]
    latest = all_df.sort_values("fetched_at_utc").drop_duplicates(subset=key, keep="last")
    latest_path = LATEST_DIR / f"{provider_slug}_latest.csv"
    latest_path = _safe_to_csv(latest, latest_path)

    print(f"[{provider_slug}] snapshot -> {snap_path}\n[{provider_slug}] history  -> {hist_path}\n[{provider_slug}] latest   -> {latest_path}")
    return latest

# ---------- your original parsing (unchanged) ----------
def find_price_strict(text: str):
    """Match $X/hr, $X per hour, $X/hour, case-insensitive."""
    if not text: return None
    m = re.search(r"\$([0-9]+(?:\.[0-9]+)?)\s*(?:/|\s*per\s*)?\s*(?:h|hr|hour)\b", text, flags=re.I)
    return float(m.group(1)) if m else None

def parse_nebius_from_html(html: str) -> dict:
    """Return {'H100': price, 'H200': price} if found."""
    soup = BeautifulSoup(html, "html.parser")
    results = {}

    # Focus on plausible pricing containers first (tables / pricing sections)
    blocks = []
    blocks.extend(soup.find_all("table"))
    if not blocks:
        blocks.extend(soup.find_all(["section","div"], class_=re.compile("price|pricing|compute", re.I)))
    if not blocks:
        blocks = soup.find_all(["div","tr","li","p","span"])

    for blk in blocks:
        t = blk.get_text(" ", strip=True)
        if not t: continue

        has_h100 = bool(re.search(r"\bH100\b", t, flags=re.I))
        has_h200 = bool(re.search(r"\bH200\b", t, flags=re.I))
        if not (has_h100 or has_h200): continue

        price = find_price_strict(t)
        if price is None or not (MIN_PRICE <= price <= MAX_PRICE): continue

        if has_h100 and "H100" not in results:
            results["H100"] = price
        if has_h200 and "H200" not in results:
            results["H200"] = price

        if len(results) == 2:
            break

    return results

# ---------- run (your flow) ----------
html = None
try:
    r = requests.get(url, headers=UA, timeout=30)
    if r.status_code == 200 and r.text:
        html = r.text
except Exception:
    html = None

results = {}
if html:
    results = parse_nebius_from_html(html)

# Optional Playwright fallback if nothing found
if not results:
    try:
        from playwright.sync_api import sync_playwright
        with sync_playwright() as p:
            browser = p.chromium.launch(headless=True)
            page = browser.new_page()
            page.goto(url, wait_until="networkidle", timeout=60000)
            page.wait_for_timeout(2000)  # allow dynamic content
            html_pw = page.content()
            browser.close()
        results = parse_nebius_from_html(html_pw)
    except Exception:
        pass  # proceed with whatever we have

# ---------- map YOUR results -> SLIM schema & save ----------
rows = []
ts = datetime.now(TZ).isoformat()
for gpu, price in results.items():
    rows.append({
        "provider": "Nebius",
        "region": "Global",            # keep simple; refine if you later detect regions
        "gpu_model": gpu,              # map gpu_type -> gpu_model
        "type": "On-Demand",
        "duration": "1h",
        "gpu_count": None,
        "price_hourly_usd": price,     # map on_demand_price -> price_hourly_usd
        "source_url": url,
        "fetched_at_utc": ts,          # map scraped_at -> fetched_at_utc
    })

df_nebius = pd.DataFrame(rows, columns=SLIM_COLS)
latest = _save_provider(df_nebius, "nebius")
print(f"Nebius rows this run: {len(df_nebius)}")
display(df_nebius.head())


[nebius] snapshot -> docs/data/snapshots/20250910_122157_nebius.csv
[nebius] history  -> docs/data/history/nebius_history.csv
[nebius] latest   -> docs/data/latest/nebius_latest.csv
Nebius rows this run: 2


,provider,region,gpu_model,type,duration,gpu_count,price_hourly_usd,source_url,fetched_at_utc
0,Nebius,Global,H200,On-Demand,1h,None,2.3,https://nebius.com/prices,2025-09-10T12:21:57.425341+00:00
1,Nebius,Global,H100,On-Demand,1h,None,2.0,https://nebius.com/prices,2025-09-10T12:21:57.425341+00:00


In [98]:
# ================= VoltagePark (async) — slim schema + per-provider history =================
# Slim schema: provider, region, gpu_model, type, duration, gpu_count,
#              price_hourly_usd, source_url, fetched_at_utc
# Py 3.8 compatible

import re, asyncio, tempfile, pandas as pd
from datetime import datetime, timezone
from pathlib import Path
from typing import Optional
from playwright.async_api import async_playwright

# ---- storage + schema helpers (same as other providers) ----
SLIM_COLS = [
    "provider","region","gpu_model","type","duration","gpu_count",
    "price_hourly_usd","source_url","fetched_at_utc"
]
BASE = Path("docs/data")
HIST_DIR = BASE / "history"
SNAP_DIR = BASE / "snapshots"
LATEST_DIR = BASE / "latest"
for d in (HIST_DIR, SNAP_DIR, LATEST_DIR):
    try:
        d.mkdir(parents=True, exist_ok=True)
    except Exception:
        pass  # fallback handled in _safe_to_csv

def _now_iso() -> str:
    return datetime.now(timezone.utc).isoformat(timespec="seconds")

def _ensure_slim(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in SLIM_COLS:
        if c not in out.columns:
            out[c] = None
    out["price_hourly_usd"] = pd.to_numeric(out["price_hourly_usd"], errors="coerce")
    out["fetched_at_utc"] = pd.to_datetime(out["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
    return out[SLIM_COLS]

def _safe_to_csv(df: pd.DataFrame, path: Path):
    try:
        path.parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(path, index=False)
        return path
    except Exception:
        tmp = Path(tempfile.gettempdir()) / path.name
        df.to_csv(tmp, index=False)
        return tmp

def _save_provider(df: pd.DataFrame, provider_slug: str):
    df = _ensure_slim(df)
    ts = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%S")

    # snapshot
    snap_path = SNAP_DIR / f"{ts}_{provider_slug}.csv"
    snap_path = _safe_to_csv(df, snap_path)

    # history (append + dedupe)
    hist_path = HIST_DIR / f"{provider_slug}_history.csv"
    if hist_path.exists():
        old = pd.read_csv(hist_path, low_memory=False)
        old["fetched_at_utc"] = pd.to_datetime(old["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
        all_df = pd.concat([old, df], ignore_index=True)
    else:
        all_df = df.copy()
    all_df = (
        all_df.dropna(subset=["gpu_model","price_hourly_usd"])
              .drop_duplicates(subset=["provider","region","gpu_model","type","duration",
                                       "fetched_at_utc","price_hourly_usd"], keep="last")
              .sort_values("fetched_at_utc")
    )
    hist_path = _safe_to_csv(all_df, hist_path)

    # latest (newest per gpu/type/region/duration)
    key = ["gpu_model","type","region","duration"]
    latest = all_df.sort_values("fetched_at_utc").drop_duplicates(subset=key, keep="last")
    latest_path = LATEST_DIR / f"{provider_slug}_latest.csv"
    latest_path = _safe_to_csv(latest, latest_path)

    print(f"[{provider_slug}] snapshot -> {snap_path}\n[{provider_slug}] history  -> {hist_path}\n[{provider_slug}] latest   -> {latest_path}")
    return latest

# ---- YOUR scraping logic, adapted to slim schema ----
async def scrape_voltagepark() -> pd.DataFrame:
    url = "https://dashboard.voltagepark.com/order/configure-deployment"
    rows = []
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, timeout=60000)
        await page.wait_for_timeout(5000)

        html = await page.content()
        await browser.close()

    for line in html.splitlines():
        if ("H100" in line or "H200" in line) and "$" in line:
            try:
                # your original pattern
                m = re.search(r"\$?(\d+(?:\.\d+)?)(?=/GPU/hour)", line)
                if m:
                    price = float(m.group(1))
                    gpu = "H100" if "H100" in line else "H200"
                    rows.append({
                        "provider": "VoltagePark",
                        "region": "US",
                        "gpu_model": gpu,
                        "type": "On-Demand",
                        "duration": "1h",
                        "gpu_count": None,
                        "price_hourly_usd": price,
                        "source_url": url,
                        "fetched_at_utc": _now_iso(),
                    })
            except Exception as e:
                # keep silent in prod; print minimal context if you want
                # print(f"[VoltagePark Parse Error] {e}")
                pass

    df = pd.DataFrame(rows, columns=SLIM_COLS)
    if df.empty:
        return _ensure_slim(df)

    # dedupe by (gpu_model, price)
    df = (df.sort_values(["gpu_model","price_hourly_usd","fetched_at_utc"])
            .drop_duplicates(subset=["gpu_model","price_hourly_usd"], keep="last")
            .reset_index(drop=True))

    # sanity: plausible $/hr range
    df = df[(df["price_hourly_usd"] > 0) & (df["price_hourly_usd"] < 200)]
    return _ensure_slim(df)

# ---- runner that works in notebooks & scripts ----
def arun(coro):
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:
        return asyncio.run(coro)
    else:
        import nest_asyncio; nest_asyncio.apply()
        return loop.run_until_complete(coro)

# ------------------------------ RUN --------------------------------
df_voltage = arun(scrape_voltagepark())
_save_provider(df_voltage, "voltagepark")
display(df_voltage.head())


[voltagepark] snapshot -> docs/data/snapshots/20250910_122207_voltagepark.csv
[voltagepark] history  -> docs/data/history/voltagepark_history.csv
[voltagepark] latest   -> docs/data/latest/voltagepark_latest.csv


,provider,region,gpu_model,type,duration,gpu_count,price_hourly_usd,source_url,fetched_at_utc
0,VoltagePark,US,H100,On-Demand,1h,None,1.99,https://dashboard.voltagepark.com/order/config...,2025-09-10 12:22:07


In [99]:
# ================= Vast.ai (async) — slim schema + per-provider history =================
# Slim schema: provider, region, gpu_model, type, duration, gpu_count,
#              price_hourly_usd, source_url, fetched_at_utc
# Py 3.8 compatible

import re, asyncio, pandas as pd, tempfile
from datetime import datetime, timezone
from pathlib import Path
from playwright.async_api import async_playwright

# -------- storage + schema helpers (same as other providers) --------
SLIM_COLS = [
    "provider","region","gpu_model","type","duration","gpu_count",
    "price_hourly_usd","source_url","fetched_at_utc"
]
BASE = Path("docs/data")
HIST_DIR = BASE / "history"
SNAP_DIR = BASE / "snapshots"
LATEST_DIR = BASE / "latest"
for d in (HIST_DIR, SNAP_DIR, LATEST_DIR):
    try:
        d.mkdir(parents=True, exist_ok=True)
    except Exception:
        pass  # fallback handled below

def _now_iso() -> str:
    return datetime.now(timezone.utc).isoformat(timespec="seconds")

def _ensure_slim(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in SLIM_COLS:
        if c not in out.columns:
            out[c] = None
    out["price_hourly_usd"] = pd.to_numeric(out["price_hourly_usd"], errors="coerce")
    out["fetched_at_utc"] = pd.to_datetime(out["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
    return out[SLIM_COLS]

def _safe_to_csv(df: pd.DataFrame, path: Path):
    try:
        path.parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(path, index=False)
        return path
    except Exception:
        tmp = Path(tempfile.gettempdir()) / path.name
        df.to_csv(tmp, index=False)
        return tmp

def _save_provider(df: pd.DataFrame, provider_slug: str):
    df = _ensure_slim(df)
    ts = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%S")

    # snapshot
    snap_path = SNAP_DIR / f"{ts}_{provider_slug}.csv"
    snap_path = _safe_to_csv(df, snap_path)

    # history (append + dedupe)
    hist_path = HIST_DIR / f"{provider_slug}_history.csv"
    if hist_path.exists():
        old = pd.read_csv(hist_path, low_memory=False)
        old["fetched_at_utc"] = pd.to_datetime(old["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
        all_df = pd.concat([old, df], ignore_index=True)
    else:
        all_df = df.copy()
    all_df = (
        all_df.dropna(subset=["gpu_model","price_hourly_usd"])
              .drop_duplicates(subset=["provider","region","gpu_model","type","duration",
                                       "fetched_at_utc","price_hourly_usd"], keep="last")
              .sort_values("fetched_at_utc")
    )
    hist_path = _safe_to_csv(all_df, hist_path)

    # latest (newest per gpu/type/region/duration)
    key = ["gpu_model","type","region","duration"]
    latest = all_df.sort_values("fetched_at_utc").drop_duplicates(subset=key, keep="last")
    latest_path = LATEST_DIR / f"{provider_slug}_latest.csv"
    latest_path = _safe_to_csv(latest, latest_path)

    print(f"[{provider_slug}] snapshot -> {snap_path}\n[{provider_slug}] history  -> {hist_path}\n[{provider_slug}] latest   -> {latest_path}")
    return latest

# -------- YOUR scraping logic, adapted to slim schema --------
async def scrape_vast_products() -> pd.DataFrame:
    url = "https://vast.ai/products/gpu-cloud"
    rows = []
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, timeout=60000)
        # Try to reveal lazy content
        await page.evaluate("window.scrollTo(0, document.body.scrollHeight)")
        await page.wait_for_timeout(1500)
        await page.evaluate("window.scrollTo(0, 0)")
        await page.wait_for_timeout(500)

        content = await page.content()
        await browser.close()

    # Your original approach: scan lines and pick $ numbers near H100/H200
    for line in content.splitlines():
        if ("H100" in line or "H200" in line) and "$" in line:
            try:
                gpu_model = "H100" if "H100" in line else "H200"
                # pull all $-bearing tokens in the line
                dollars = [s for s in re.split(r"\s+", line) if "$" in s]
                price_val = None
                for token in dollars:
                    clean = "".join(c for c in token if c.isdigit() or c == ".")
                    if not clean:
                        continue
                    price = float(clean)
                    if 0.1 < price < 100:  # sanity filter like you had
                        price_val = price
                        break
                if price_val is None:
                    continue

                rows.append({
                    "provider": "Vast.ai",
                    "region": "Global",
                    "gpu_model": gpu_model,
                    "type": "On-Demand",
                    "duration": "1h",
                    "gpu_count": None,
                    "price_hourly_usd": price_val,
                    "source_url": url,
                    "fetched_at_utc": _now_iso(),
                })
            except Exception:
                # swallow parse errors to keep the run clean
                pass

    df = pd.DataFrame(rows, columns=SLIM_COLS)
    if df.empty:
        return _ensure_slim(df)

    # Deduplicate by (gpu_model, price)
    df = (df.sort_values(["gpu_model","price_hourly_usd","fetched_at_utc"])
            .drop_duplicates(subset=["gpu_model","price_hourly_usd"], keep="last")
            .reset_index(drop=True))
    # Sanity clamp
    df = df[(df["price_hourly_usd"] > 0) & (df["price_hourly_usd"] < 200)]
    return _ensure_slim(df)

def arun(coro):
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:
        return asyncio.run(coro)
    else:
        import nest_asyncio; nest_asyncio.apply()
        return loop.run_until_complete(coro)

df_vastp = arun(scrape_vast_products())
_save_provider(df_vastp, "vastai")
display(df_vastp.head())


[vastai] snapshot -> docs/data/snapshots/20250910_122212_vastai.csv
[vastai] history  -> docs/data/history/vastai_history.csv
[vastai] latest   -> docs/data/latest/vastai_latest.csv


,provider,region,gpu_model,type,duration,gpu_count,price_hourly_usd,source_url,fetched_at_utc
0,Vast.ai,Global,H100,On-Demand,1h,None,1.25,https://vast.ai/products/gpu-cloud,2025-09-10 12:22:12


In [100]:
# ==== Shadeform: precise matcher (nearest-price + hourly hint) ====

import re, asyncio, pandas as pd, tempfile
from datetime import datetime, timezone
from pathlib import Path
from playwright.async_api import async_playwright

# slim schema storage helpers (use the same ones you already have)
SLIM_COLS = [
    "provider","region","gpu_model","type","duration","gpu_count",
    "price_hourly_usd","source_url","fetched_at_utc"
]

def _now_iso():
    return datetime.now(timezone.utc).isoformat(timespec="seconds")

def _ensure_slim(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in SLIM_COLS:
        if c not in out.columns: out[c] = None
    out["price_hourly_usd"] = pd.to_numeric(out["price_hourly_usd"], errors="coerce")
    out["fetched_at_utc"] = pd.to_datetime(out["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
    return out[SLIM_COLS]

# --------- robust extractors ----------
# require an hourly hint, allowing variants like "/GPU/hour"
PRICE_RE = re.compile(
    r"\$\s*([0-9]+(?:\.[0-9]+)?)\s*(?:/GPU)?\s*(?:/|\s*(?:per|an)\s*)?(?:h|hr|hour)\b",
    re.I
)

GPU_TOKENS = {
    "H100": re.compile(r"\bH100\b", re.I),
    "H200": re.compile(r"\bH200\b", re.I),
    # include B200 so we don't steal its prices
    "_OTHER": re.compile(r"\b(?:B200|H800|A100|A800)\b", re.I),
}

def _find_token_positions(text: str):
    positions = {k: [] for k in GPU_TOKENS.keys()}
    for name, pat in GPU_TOKENS.items():
        for m in pat.finditer(text):
            positions[name].append(m.start())
    return positions

def _find_price_positions(text: str):
    return [(float(m.group(1)), m.start()) for m in PRICE_RE.finditer(text)]

def _nearest_price_to_token(text: str, token: str, window: int = 220):
    """Yield (model, price) pairs by attaching each token occurrence
       to the nearest price with an hourly hint, only if it is closer
       to this token than to any other GPU token."""
    tok_positions = _find_token_positions(text)
    prices = _find_price_positions(text)
    if not tok_positions.get(token) or not prices:
        return []

    # all GPU-ish positions (to compete for 'closeness')
    competitor_positions = []
    for k, pos_list in tok_positions.items():
        if k == token:  # we compare against others later
            continue
        competitor_positions.extend(pos_list)

    rows = []
    for gpos in tok_positions[token]:
        # candidates within a window around the GPU string
        cands = [(price, ppos, abs(ppos - gpos)) for (price, ppos) in prices if abs(ppos - gpos) <= window]
        if not cands:
            continue
        # pick nearest price to this token
        price, ppos, dist = min(cands, key=lambda t: t[2])

        # ensure this price isn't actually closer to another GPU token (e.g., B200)
        if competitor_positions:
            nearest_other = min(abs(ppos - op) for op in competitor_positions)
            if nearest_other < dist:
                continue  # skip: price belongs to another GPU mention

        rows.append((token, price))
    return rows

# --------- scraper ----------
async def scrape_shadeform_rich() -> pd.DataFrame:
    url = "https://www.shadeform.ai/"
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, timeout=60000, wait_until="domcontentloaded")
        await page.evaluate("window.scrollTo(0, document.body.scrollHeight)")
        await page.wait_for_timeout(900)
        body = await page.inner_text("body")
        await browser.close()

    body = re.sub(r"\s+", " ", body)

    rows = []
    for gpu in ("H100", "H200"):
        for model, price in _nearest_price_to_token(body, gpu, window=220):
            # sanity clamp to avoid accidental captures (tune if needed)
            if not (0.25 <= price <= 8.0):
                continue
            rows.append({
                "provider": "Shadeform",
                "region": "Global",
                "gpu_model": model,
                "type": "On-Demand",
                "duration": "1h",
                "gpu_count": None,
                "price_hourly_usd": price,
                "source_url": url,
                "fetched_at_utc": _now_iso(),
            })

    df = pd.DataFrame(rows, columns=SLIM_COLS)
    if df.empty:
        return _ensure_slim(df)

    # de-dupe (gpu_model, price)
    df = (df.sort_values(["gpu_model","price_hourly_usd","fetched_at_utc"])
            .drop_duplicates(subset=["gpu_model","price_hourly_usd"], keep="last")
            .reset_index(drop=True))
    return _ensure_slim(df)

# ---- runner ----
def arun(coro):
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:
        return asyncio.run(coro)
    else:
        import nest_asyncio; nest_asyncio.apply()
        return loop.run_until_complete(coro)

# Example:
df_shade = arun(scrape_shadeform_rich())
display(df_shade)


,provider,region,gpu_model,type,duration,gpu_count,price_hourly_usd,source_url,fetched_at_utc
0,Shadeform,Global,H100,On-Demand,1h,None,1.99,https://www.shadeform.ai/,2025-09-10 12:22:14
1,Shadeform,Global,H100,On-Demand,1h,None,2.35,https://www.shadeform.ai/,2025-09-10 12:22:14


In [101]:
# ================= Paperspace (async) — use your approach, keep correct rows =================
# Output schema (slim): provider, region, gpu_model, type, duration, gpu_count,
#                       price_hourly_usd, source_url, fetched_at_utc

import re, asyncio, pandas as pd, tempfile
from bs4 import BeautifulSoup
from datetime import datetime, timezone
from pathlib import Path
from playwright.async_api import async_playwright

# ---- storage helpers (same as other providers) ----
SLIM_COLS = [
    "provider","region","gpu_model","type","duration","gpu_count",
    "price_hourly_usd","source_url","fetched_at_utc"
]
BASE = Path("docs/data"); HIST_DIR = BASE/"history"; SNAP_DIR = BASE/"snapshots"; LATEST_DIR = BASE/"latest"
for d in (HIST_DIR, SNAP_DIR, LATEST_DIR):
    try: d.mkdir(parents=True, exist_ok=True)
    except Exception: pass

def _now_iso(): return datetime.now(timezone.utc).isoformat(timespec="seconds")

def _ensure_slim(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in SLIM_COLS:
        if c not in out.columns: out[c] = None
    out["price_hourly_usd"] = pd.to_numeric(out["price_hourly_usd"], errors="coerce")
    out["fetched_at_utc"] = pd.to_datetime(out["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
    return out[SLIM_COLS]

def _safe_to_csv(df: pd.DataFrame, path: Path):
    try:
        path.parent.mkdir(parents=True, exist_ok=True); df.to_csv(path, index=False); return path
    except Exception:
        tmp = Path(tempfile.gettempdir()) / path.name; df.to_csv(tmp, index=False); return tmp

def _save_provider(df: pd.DataFrame, slug: str):
    df = _ensure_slim(df); ts = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%S")
    snap = _safe_to_csv(df, SNAP_DIR/f"{ts}_{slug}.csv")
    # history
    hist = HIST_DIR/f"{slug}_history.csv"
    if hist.exists():
        old = pd.read_csv(hist, low_memory=False)
        old["fetched_at_utc"] = pd.to_datetime(old["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
        all_df = pd.concat([old, df], ignore_index=True)
    else:
        all_df = df.copy()
    all_df = (all_df.dropna(subset=["gpu_model","price_hourly_usd"])
                    .drop_duplicates(subset=["provider","region","gpu_model","type","duration",
                                             "fetched_at_utc","price_hourly_usd"], keep="last")
                    .sort_values("fetched_at_utc"))
    hist = _safe_to_csv(all_df, hist)
    # latest
    key = ["gpu_model","type","region","duration"]
    latest = all_df.sort_values("fetched_at_utc").drop_duplicates(subset=key, keep="last")
    latest_path = _safe_to_csv(latest, LATEST_DIR/f"{slug}_latest.csv")
    print(f"[{slug}] snapshot -> {snap}\n[{slug}] history  -> {hist}\n[{slug}] latest   -> {latest_path}")
    return latest

# ---- strict extractors (but still tolerant to site markup) ----
GPU_PAT = re.compile(r"(H(?:100|200)(?:\s*(?:SXM|PCIE|NVL))?(?:\s*\d{2,3}\s*GB)?)", re.I)
# require an hourly hint somewhere in the same block to avoid platform prices, etc.
PRICE_HOURLY = re.compile(r"\$\s*([0-9]+(?:\.[0-9]+)?)\s*(?:/|\s*(?:per|an)\s*)?(?:GPU\s*/\s*)?(?:h|hr|hour)\b", re.I)

async def scrape_paperspace() -> pd.DataFrame:
    url = "https://www.paperspace.com/pricing"
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, timeout=90000)               # your simple navigation
        await page.wait_for_timeout(8000)                  # your “just wait a few seconds”
        html = await page.content()
        await browser.close()

    rows = []
    soup = BeautifulSoup(html, "html.parser")
    # scan reasonable blocks; stick to your block-scan approach
    for blk in soup.find_all(["tr","div","section","article","li"], recursive=True):
        txt = blk.get_text(" ", strip=True)
        if not txt: 
            continue
        # must mention H100/H200 AND 'hour' to qualify
        if ("H100" not in txt and "H200" not in txt) or ("hour" not in txt.lower()):
            continue

        # model: first explicit H100/H200-ish token found
        mm = GPU_PAT.search(txt)
        if not mm:
            continue
        model = mm.group(1).upper()

        # price: $… with an hourly hint in the same block
        pm = PRICE_HOURLY.search(txt)
        if not pm:
            continue
        price = float(pm.group(1))
        # sanity band to drop weird captures
        if not (0.2 <= price <= 50.0):
            continue

        rows.append({
            "provider": "Paperspace",
            "region": "Global",
            "gpu_model": model,          # "H100", "H100 PCIE 80GB", etc.
            "type": "On-Demand",
            "duration": "1h",
            "gpu_count": None,
            "price_hourly_usd": price,
            "source_url": url,
            "fetched_at_utc": _now_iso(),
        })

    df = pd.DataFrame(rows, columns=SLIM_COLS)
    if df.empty:
        return _ensure_slim(df)

    # de-dupe (gpu_model, price)
    df = (df.sort_values(["gpu_model","price_hourly_usd","fetched_at_utc"])
            .drop_duplicates(subset=["gpu_model","price_hourly_usd"], keep="last")
            .reset_index(drop=True))
    return _ensure_slim(df)

# ---- runner that works in both scripts & notebooks ----
def arun(coro):
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:
        return asyncio.run(coro)
    else:
        import nest_asyncio; nest_asyncio.apply()
        return loop.run_until_complete(coro)

# ------------------------------ RUN --------------------------------
df_paperspace = arun(scrape_paperspace())
latest_paperspace = _save_provider(df_paperspace, "paperspace")
display(df_paperspace.head(20))


[paperspace] snapshot -> docs/data/snapshots/20250910_122224_paperspace.csv
[paperspace] history  -> docs/data/history/paperspace_history.csv
[paperspace] latest   -> docs/data/latest/paperspace_latest.csv


,provider,region,gpu_model,type,duration,gpu_count,price_hourly_usd,source_url,fetched_at_utc
0,Paperspace,Global,H100,On-Demand,1h,None,2.24,https://www.paperspace.com/pricing,2025-09-10 12:22:24


In [102]:
# ================= TensorDock H100 (static) — slim schema + per-provider history =================
# Output schema: provider, region, gpu_model, type, duration, gpu_count,
#                price_hourly_usd, source_url, fetched_at_utc
# Py 3.8 compatible

import re, requests, pandas as pd, tempfile
from bs4 import BeautifulSoup
from datetime import datetime, timezone
from pathlib import Path

# -------- pages & patterns (from your code) --------
PAGES = [
    "https://tensordock.com/gpu-h100",
    "https://tensordock.com/cloud-gpus",
    "https://tensordock.com/comparison-gcp",
]
PATTERNS = [
    re.compile(r"H100.*?\$([0-9]+(?:\.[0-9]+)?)\s*/?\s*hr", re.I|re.S),
    re.compile(r"from\s*\$([0-9]+(?:\.[0-9]+)?)\s*/?\s*hr.*?H100", re.I|re.S),
    re.compile(r"\$([0-9]+(?:\.[0-9]+)?)\s*/?\s*hour.*?H100", re.I|re.S),
]
HEADERS = {"User-Agent": "Mozilla/5.0"}

# -------- storage + schema helpers (same as other providers) --------
SLIM_COLS = [
    "provider","region","gpu_model","type","duration","gpu_count",
    "price_hourly_usd","source_url","fetched_at_utc"
]
BASE = Path("docs/data")
HIST_DIR = BASE / "history"
SNAP_DIR = BASE / "snapshots"
LATEST_DIR = BASE / "latest"
for d in (HIST_DIR, SNAP_DIR, LATEST_DIR):
    try:
        d.mkdir(parents=True, exist_ok=True)
    except Exception:
        pass  # fall back handled in _safe_to_csv

def _now_iso() -> str:
    return datetime.now(timezone.utc).isoformat(timespec="seconds")

def _ensure_slim(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in SLIM_COLS:
        if c not in out.columns: out[c] = None
    out["price_hourly_usd"] = pd.to_numeric(out["price_hourly_usd"], errors="coerce")
    out["fetched_at_utc"] = pd.to_datetime(out["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
    return out[SLIM_COLS]

def _safe_to_csv(df: pd.DataFrame, path: Path):
    try:
        path.parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(path, index=False)
        return path
    except Exception:
        tmp = Path(tempfile.gettempdir()) / path.name
        df.to_csv(tmp, index=False)
        return tmp

def _save_provider(df: pd.DataFrame, provider_slug: str):
    df = _ensure_slim(df)
    ts = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%S")
    # snapshot
    snap_path = _safe_to_csv(df, SNAP_DIR / f"{ts}_{provider_slug}.csv")
    # history (append + dedupe)
    hist_path = HIST_DIR / f"{provider_slug}_history.csv"
    if hist_path.exists():
        old = pd.read_csv(hist_path, low_memory=False)
        old["fetched_at_utc"] = pd.to_datetime(old["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
        all_df = pd.concat([old, df], ignore_index=True)
    else:
        all_df = df.copy()
    all_df = (all_df
              .dropna(subset=["gpu_model","price_hourly_usd"])
              .drop_duplicates(subset=["provider","region","gpu_model","type","duration",
                                       "fetched_at_utc","price_hourly_usd"], keep="last")
              .sort_values("fetched_at_utc"))
    hist_path = _safe_to_csv(all_df, hist_path)
    # latest (newest per gpu/type/region/duration)
    key = ["gpu_model","type","region","duration"]
    latest = all_df.sort_values("fetched_at_utc").drop_duplicates(subset=key, keep="last")
    latest_path = _safe_to_csv(latest, LATEST_DIR / f"{provider_slug}_latest.csv")
    print(f"[{provider_slug}] snapshot -> {snap_path}\n[{provider_slug}] history  -> {hist_path}\n[{provider_slug}] latest   -> {latest_path}")
    return latest

# -------- scraper (uses your logic, mapped to slim schema) --------
def scrape_tensordock_public_h100() -> pd.DataFrame:
    rows = []
    for url in PAGES:
        try:
            r = requests.get(url, headers=HEADERS, timeout=30)
            r.raise_for_status()
            soup = BeautifulSoup(r.text, "html.parser")
            text = soup.get_text(" ", strip=True)
            price = None
            for pat in PATTERNS:
                m = pat.search(text)
                if m:
                    price = float(m.group(1))
                    break
            if price and (0.2 <= price <= 50.0):  # sanity band for $/GPU/hr
                rows.append({
                    "provider": "TensorDock",
                    "region": "Global",
                    "gpu_model": "H100",
                    "type": "On-Demand",
                    "duration": "1h",
                    "gpu_count": 1,
                    "price_hourly_usd": price,
                    "source_url": url,
                    "fetched_at_utc": _now_iso(),
                })
        except Exception as e:
            print(f"[TensorDock] {url} -> {e}")

    if not rows:
        return _ensure_slim(pd.DataFrame(columns=SLIM_COLS))

    # Deduplicate: keep the **lowest** "from" price across pages
    df = pd.DataFrame(rows)
    df = (df.sort_values("price_hourly_usd")
            .drop_duplicates(subset=["provider","gpu_model"], keep="first")
            .reset_index(drop=True))
    return _ensure_slim(df)

# ------------------------------ RUN --------------------------------
df_tensordock = scrape_tensordock_public_h100()
latest_tensordock = _save_provider(df_tensordock, "tensordock")
print(df_tensordock)


[tensordock] snapshot -> docs/data/snapshots/20250910_122225_tensordock.csv
[tensordock] history  -> docs/data/history/tensordock_history.csv
[tensordock] latest   -> docs/data/latest/tensordock_latest.csv
     provider  region gpu_model       type duration  gpu_count  \
0  TensorDock  Global      H100  On-Demand       1h          1   

   price_hourly_usd                       source_url      fetched_at_utc  
0              2.25  https://tensordock.com/gpu-h100 2025-09-10 12:22:24  


In [103]:
# ============== Hydra Host (Brokkr) — slim schema + per-provider history ==============
# Output schema: provider, region, gpu_model, type, duration, gpu_count,
#                price_hourly_usd, source_url, fetched_at_utc
# Py 3.8 compatible

import re, asyncio, pandas as pd, tempfile
from datetime import datetime, timezone
from pathlib import Path
from playwright.async_api import async_playwright

# -------- storage + schema helpers (same pattern as other providers) --------
SLIM_COLS = [
    "provider","region","gpu_model","type","duration","gpu_count",
    "price_hourly_usd","source_url","fetched_at_utc"
]
BASE = Path("docs/data")
HIST_DIR = BASE / "history"
SNAP_DIR = BASE / "snapshots"
LATEST_DIR = BASE / "latest"
for d in (HIST_DIR, SNAP_DIR, LATEST_DIR):
    try: d.mkdir(parents=True, exist_ok=True)
    except Exception: pass

def _now_iso() -> str:
    return datetime.now(timezone.utc).isoformat(timespec="seconds")

def _ensure_slim(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in SLIM_COLS:
        if c not in out.columns: out[c] = None
    out["price_hourly_usd"] = pd.to_numeric(out["price_hourly_usd"], errors="coerce")
    out["fetched_at_utc"] = pd.to_datetime(out["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
    return out[SLIM_COLS]

def _safe_to_csv(df: pd.DataFrame, path: Path):
    try:
        path.parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(path, index=False)
        return path
    except Exception:
        tmp = Path(tempfile.gettempdir()) / path.name
        df.to_csv(tmp, index=False)
        return tmp

def _save_provider(df: pd.DataFrame, provider_slug: str):
    df = _ensure_slim(df)
    ts = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%S")
    # snapshot
    snap_path = _safe_to_csv(df, SNAP_DIR / f"{ts}_{provider_slug}.csv")
    # history (append + dedupe)
    hist_path = HIST_DIR / f"{provider_slug}_history.csv"
    if hist_path.exists():
        old = pd.read_csv(hist_path, low_memory=False)
        old["fetched_at_utc"] = pd.to_datetime(old["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
        all_df = pd.concat([old, df], ignore_index=True)
    else:
        all_df = df.copy()
    all_df = (
        all_df.dropna(subset=["gpu_model","price_hourly_usd"])
              .drop_duplicates(subset=["provider","region","gpu_model","type","duration",
                                       "fetched_at_utc","price_hourly_usd"], keep="last")
              .sort_values("fetched_at_utc")
    )
    hist_path = _safe_to_csv(all_df, hist_path)
    # latest (newest per gpu/type/region/duration)
    key = ["gpu_model","type","region","duration"]
    latest = all_df.sort_values("fetched_at_utc").drop_duplicates(subset=key, keep="last")
    latest_path = _safe_to_csv(latest, LATEST_DIR / f"{provider_slug}_latest.csv")
    print(f"[{provider_slug}] snapshot -> {snap_path}\n[{provider_slug}] history  -> {hist_path}\n[{provider_slug}] latest   -> {latest_path}")
    return latest

# -------- your Brokkr scraper, tightened to only accept "per card-hour" prices --------
GPU_RE = r"(H100|H200)"
PRICE_PER_CARDHR = r"\$\s*([0-9]+(?:\.[0-9]+)?)\s*(?:per\s*card[-\s]?hour|/card[-\s]?hour)\b"
PATS = [
    re.compile(rf"{GPU_RE}.{{0,220}}?{PRICE_PER_CARDHR}", re.I | re.S),
    re.compile(rf"{PRICE_PER_CARDHR}.{{0,220}}?{GPU_RE}", re.I | re.S),
]

async def scrape_brokkr() -> pd.DataFrame:
    url = "https://brokkr.hydrahost.com/inventory"
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, timeout=90000, wait_until="domcontentloaded")
        # help hydrate lazy content
        for _ in range(2):
            await page.evaluate("window.scrollBy(0, document.body.scrollHeight)")
            await page.wait_for_timeout(700)
        body_text = await page.inner_text("body")
        await browser.close()

    text = re.sub(r"\s+", " ", body_text)
    rows = []

    for pat in PATS:
        for m in pat.finditer(text):
            # Depending on which pattern matched, group order differs
            groups = m.groups()
            # Normalize extraction: model + price are always present
            if len(groups) == 2:
                # pattern 1: (GPU, price)
                gpu_model, price_str = groups
            elif len(groups) == 3:
                # pattern 2 returns (price, GPU) because of nested groups; pick numeric+gpu
                # groups could be ('12.34', 'H100') or ('12.34', 'card-hour', 'H100') depending on regex engine
                nums = [g for g in groups if g and re.fullmatch(r"[0-9]+(?:\.[0-9]+)?", g)]
                gpus = [g for g in groups if g and re.fullmatch(r"H100|H200", g, flags=re.I)]
                if not nums or not gpus:
                    continue
                price_str, gpu_model = nums[0], gpus[0]
            else:
                # Safe fallback: find first number and first GPU token in the match
                seg = m.group(0)
                pm = re.search(r"[0-9]+(?:\.[0-9]+)?", seg)
                gm = re.search(r"H100|H200", seg, flags=re.I)
                if not (pm and gm):
                    continue
                price_str, gpu_model = pm.group(0), gm.group(0)

            try:
                price = float(price_str)
            except Exception:
                continue

            # sanity band for per-card hour pricing
            if not (0.2 <= price <= 50.0):
                continue

            rows.append({
                "provider": "Hydra Host (Brokkr)",
                "region": "Global",
                "gpu_model": gpu_model.upper(),
                "type": "On-Demand",
                "duration": "1h",
                "gpu_count": 1,
                "price_hourly_usd": price,   # per card-hour = per-GPU hourly
                "source_url": url,
                "fetched_at_utc": _now_iso(),
            })

    df = pd.DataFrame(rows, columns=SLIM_COLS)
    if df.empty:
        return _ensure_slim(df)

    # de-dupe (gpu_model, price)
    df = (df.sort_values(["gpu_model","price_hourly_usd","fetched_at_utc"])
            .drop_duplicates(subset=["gpu_model","price_hourly_usd"], keep="last")
            .reset_index(drop=True))
    return _ensure_slim(df)

# -------- runner that works in both notebooks & scripts --------
def arun(coro):
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:
        return asyncio.run(coro)
    else:
        import nest_asyncio; nest_asyncio.apply()
        return loop.run_until_complete(coro)

# ------------------------------ RUN --------------------------------
df_brokkr = arun(scrape_brokkr())
latest_brokkr = _save_provider(df_brokkr, "brokkr")
print(df_brokkr)
display(df_brokkr.head())


[brokkr] snapshot -> docs/data/snapshots/20250910_122229_brokkr.csv
[brokkr] history  -> docs/data/history/brokkr_history.csv
[brokkr] latest   -> docs/data/latest/brokkr_latest.csv
              provider  region gpu_model       type duration  gpu_count  \
0  Hydra Host (Brokkr)  Global      H100  On-Demand       1h          1   
1  Hydra Host (Brokkr)  Global      H200  On-Demand       1h          1   

   price_hourly_usd                              source_url  \
0               2.3  https://brokkr.hydrahost.com/inventory   
1               2.5  https://brokkr.hydrahost.com/inventory   

       fetched_at_utc  
0 2025-09-10 12:22:28  
1 2025-09-10 12:22:28  


,provider,region,gpu_model,type,duration,gpu_count,price_hourly_usd,source_url,fetched_at_utc
0,Hydra Host (Brokkr),Global,H100,On-Demand,1h,1,2.3,https://brokkr.hydrahost.com/inventory,2025-09-10 12:22:28
1,Hydra Host (Brokkr),Global,H200,On-Demand,1h,1,2.5,https://brokkr.hydrahost.com/inventory,2025-09-10 12:22:28


In [104]:
# ============ Crusoe Cloud (async) — table scrape → slim schema + history ============

import re, asyncio, pandas as pd, tempfile
from datetime import datetime, timezone
from pathlib import Path
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup

# ---------- slim schema + storage helpers (same as other providers) ----------
SLIM_COLS = [
    "provider","region","gpu_model","type","duration","gpu_count",
    "price_hourly_usd","source_url","fetched_at_utc"
]
BASE = Path("docs/data")
HIST_DIR = BASE / "history"
SNAP_DIR = BASE / "snapshots"
LATEST_DIR = BASE / "latest"
for d in (HIST_DIR, SNAP_DIR, LATEST_DIR):
    try: d.mkdir(parents=True, exist_ok=True)
    except Exception: pass

def _now_iso() -> str:
    return datetime.now(timezone.utc).isoformat(timespec="seconds")

def _ensure_slim(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in SLIM_COLS:
        if c not in out.columns: out[c] = None
    out["price_hourly_usd"] = pd.to_numeric(out["price_hourly_usd"], errors="coerce")
    out["fetched_at_utc"] = pd.to_datetime(out["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
    return out[SLIM_COLS]

def _safe_to_csv(df: pd.DataFrame, path: Path):
    try:
        path.parent.mkdir(parents=True, exist_ok=True); df.to_csv(path, index=False); return path
    except Exception:
        tmp = Path(tempfile.gettempdir()) / path.name; df.to_csv(tmp, index=False); return tmp

def _save_provider(df: pd.DataFrame, slug: str):
    df = _ensure_slim(df)
    ts = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%S")
    snap = _safe_to_csv(df, SNAP_DIR / f"{ts}_{slug}.csv")
    # history append+dedupe
    hist = HIST_DIR / f"{slug}_history.csv"
    if hist.exists():
        old = pd.read_csv(hist, low_memory=False)
        old["fetched_at_utc"] = pd.to_datetime(old["fetched_at_utc"], errors="coerce", utc=True).dt.tz_convert(None)
        all_df = pd.concat([old, df], ignore_index=True)
    else:
        all_df = df.copy()
    all_df = (all_df
        .dropna(subset=["gpu_model","price_hourly_usd"])
        .drop_duplicates(subset=["provider","region","gpu_model","type","duration",
                                 "fetched_at_utc","price_hourly_usd"], keep="last")
        .sort_values("fetched_at_utc"))
    hist = _safe_to_csv(all_df, hist)
    # latest per gpu/type/region/duration
    key = ["gpu_model","type","region","duration"]
    latest = all_df.sort_values("fetched_at_utc").drop_duplicates(subset=key, keep="last")
    latest_path = _safe_to_csv(latest, LATEST_DIR / f"{slug}_latest.csv")
    print(f"[{slug}] snapshot -> {snap}\n[{slug}] history  -> {hist}\n[{slug}] latest   -> {latest_path}")
    return latest

# ---------- parsing helpers ----------
PRICE_RE = re.compile(r"\$?\s*([0-9]+(?:\.[0-9]+)?)\s*(?:/|\s*per\s*)?\s*(?:h|hr|hour)\b", re.I)

def _parse_price(cell_text: str):
    if not cell_text: return None
    m = PRICE_RE.search(cell_text.replace(",", ""))
    if not m:
        # fallback: plain $N.NN without explicit /hr
        m2 = re.search(r"\$?\s*([0-9]+(?:\.[0-9]+)?)\b", cell_text.replace(",", ""))
        return float(m2.group(1)) if m2 else None
    return float(m.group(1))

def _is_h_model(text: str) -> bool:
    t = text.upper()
    return ("H100" in t) or ("H200" in t)

def _model_from(text: str) -> str:
    return "H100" if "H100" in text.upper() else "H200"

# ---------- scraper ----------
async def scrape_crusoe_table() -> pd.DataFrame:
    url = "https://www.crusoe.ai/cloud/pricing"
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, timeout=90000, wait_until="domcontentloaded")
        # help render
        for _ in range(2):
            await page.evaluate("window.scrollBy(0, document.body.scrollHeight)")
            await page.wait_for_timeout(800)
        html = await page.content()
        await browser.close()

    soup = BeautifulSoup(html, "html.parser")
    rows_out = []

    # Find all table rows; filter to those that mention H100/H200
    for tr in soup.find_all("tr"):
        tds = [c.get_text(" ", strip=True) for c in tr.find_all("td")]
        if not tds or not any(_is_h_model(c) for c in tds):
            continue

        model = _model_from(" ".join(tds))

        # Try to map columns conservatively:
        # Common layout: [Model, On-Demand, Spot?, Reserved 6m, Reserved 1y, Reserved 3y, ...]
        # We’ll grab by position if present, else try to read by header alignment.
        on_demand = _parse_price(tds[1]) if len(tds) > 1 else None
        res_6m    = _parse_price(tds[3]) if len(tds) > 3 else None
        res_1y    = _parse_price(tds[4]) if len(tds) > 4 else None
        res_3y    = _parse_price(tds[5]) if len(tds) > 5 else None

        # Build normalised slim rows
        if on_demand is not None:
            rows_out.append({
                "provider": "CrusoeCloud",
                "region": "Global",
                "gpu_model": model,
                "type": "On-Demand",
                "duration": "1h",
                "gpu_count": 1,
                "price_hourly_usd": on_demand,
                "source_url": url,
                "fetched_at_utc": _now_iso(),
            })
        if res_6m is not None:
            rows_out.append({
                "provider": "CrusoeCloud",
                "region": "Global",
                "gpu_model": model,
                "type": "Reserved-6m",
                "duration": "1h",
                "gpu_count": 1,
                "price_hourly_usd": res_6m,
                "source_url": url,
                "fetched_at_utc": _now_iso(),
            })
        if res_1y is not None:
            rows_out.append({
                "provider": "CrusoeCloud",
                "region": "Global",
                "gpu_model": model,
                "type": "Reserved-1y",
                "duration": "1h",
                "gpu_count": 1,
                "price_hourly_usd": res_1y,
                "source_url": url,
                "fetched_at_utc": _now_iso(),
            })
        if res_3y is not None:
            rows_out.append({
                "provider": "CrusoeCloud",
                "region": "Global",
                "gpu_model": model,
                "type": "Reserved-3y",
                "duration": "1h",
                "gpu_count": 1,
                "price_hourly_usd": res_3y,
                "source_url": url,
                "fetched_at_utc": _now_iso(),
            })

    df = pd.DataFrame(rows_out, columns=SLIM_COLS)
    if df.empty:
        return _ensure_slim(df)

    # de-dupe & sanity
    df = (df.sort_values(["gpu_model","type","price_hourly_usd","fetched_at_utc"])
            .drop_duplicates(subset=["gpu_model","type","price_hourly_usd"], keep="last")
            .reset_index(drop=True))
    # plausible hourly band
    df = df[(df["price_hourly_usd"] > 0) & (df["price_hourly_usd"] < 200)]
    return _ensure_slim(df)

# ---------- runner that works in scripts & notebooks ----------
def arun(coro):
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:
        return asyncio.run(coro)
    else:
        import nest_asyncio; nest_asyncio.apply()
        return loop.run_until_complete(coro)

# ------------------------------ RUN --------------------------------
df_crusoe = arun(scrape_crusoe_table())
latest_crusoe = _save_provider(df_crusoe, "crusoecloud")
print(df_crusoe.head(20))


[crusoecloud] snapshot -> docs/data/snapshots/20250910_122232_crusoecloud.csv
[crusoecloud] history  -> docs/data/history/crusoecloud_history.csv
[crusoecloud] latest   -> docs/data/latest/crusoecloud_latest.csv
      provider  region gpu_model         type duration  gpu_count  \
0  CrusoeCloud  Global      H100    On-Demand       1h          1   
1  CrusoeCloud  Global      H100  Reserved-1y       1h          1   
2  CrusoeCloud  Global      H100  Reserved-3y       1h          1   
3  CrusoeCloud  Global      H100  Reserved-6m       1h          1   
4  CrusoeCloud  Global      H200    On-Demand       1h          1   
5  CrusoeCloud  Global      H200  Reserved-1y       1h          1   
6  CrusoeCloud  Global      H200  Reserved-3y       1h          1   
7  CrusoeCloud  Global      H200  Reserved-6m       1h          1   

   price_hourly_usd                           source_url      fetched_at_utc  
0              3.90  https://www.crusoe.ai/cloud/pricing 2025-09-10 12:22:32  
1       

In [105]:
# OVHcloud H100/H200 — get the *correct per-GPU hourly price* from the public prices table
# - Ties each $…/hour to the same row as H100/H200
# - Extracts the GPU count from the row (1×/2×/4×/8× or “… GPUs”)
# - per_gpu = instance_price / parsed_gpu_count  (NO 8× assumption)
# - Returns both instance price and per-GPU price

import re, requests, pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

def timestamp(): return datetime.utcnow().isoformat()

USD_HOURLY = re.compile(r"\$\s*([0-9]+(?:\.[0-9]+)?)\s*(?:/|\s*(?:per|an)\s*)?(?:h|hr|hour)\b", re.I)

# GPU count detectors (row-level + cell/variant-level)
COUNT_PATS = [
    re.compile(r"(\d+)\s*[×x]\s*(?:NVIDIA\s*)?(H100|H200)\b", re.I),  # "8× H100"
    re.compile(r"(H100|H200)\s*[×x]\s*(\d+)\b", re.I),               # "H100 × 8"
    re.compile(r"(\d+)\s*(?:GPU|GPUs)\b", re.I),                     # "8 GPUs"
    re.compile(r"\b(\d+)\s*[×x]\b", re.I),                           # "4x"
]

def _parse_count(text: str):
    for pat in COUNT_PATS:
        m = pat.search(text)
        if not m: 
            continue
        for g in m.groups():
            if g and g.isdigit():
                n = int(g)
                if 1 <= n <= 16:
                    return n
    return None

def scrape_ovhcloud_correct(url="https://www.ovhcloud.com/en/public-cloud/prices/") -> pd.DataFrame:
    html = requests.get(url, headers={"User-Agent":"Mozilla/5.0"}, timeout=60).text
    soup = BeautifulSoup(html, "html.parser")

    rows = []
    for table in soup.select("table"):
        # headers so we can label which column the number came from
        headers = [th.get_text(" ", strip=True) for th in table.select("thead th")]
        if not headers:
            first = table.find("tr")
            if first:
                headers = [td.get_text(" ", strip=True) for td in first.find_all(["th","td"])]

        body_rows = table.select("tbody tr") or table.select("tr")
        for tr in body_rows:
            tds = tr.find_all("td")
            if not tds: 
                continue
            cells = [td.get_text(" ", strip=True) for td in tds]
            row_txt = " ".join(cells)
            up = row_txt.upper()
            if ("H100" not in up) and ("H200" not in up):
                continue

            model = "H100" if "H100" in up else "H200"
            row_count = _parse_count(row_txt)

            for idx, (td, cell) in enumerate(zip(tds, cells)):
                m = USD_HOURLY.search(cell)
                if not m:
                    continue
                instance_price = float(m.group(1))

                # try counts in cell and header/variant too
                var = headers[idx] if idx < len(headers) and headers else f"col_{idx+1}"
                count = (
                    _parse_count(cell) or
                    _parse_count(var)  or
                    row_count
                )
                if count is None:
                    # if we can't prove node size, skip (prevents wrong divide)
                    continue

                per_gpu = instance_price / count
                if not (0.25 <= per_gpu <= 20.0):
                    continue

                rows.append({
                    "provider": "OVHcloud",
                    "region": "Global",
                    "gpu_model": model,
                    "instance_type": f"public-cloud/{var}",
                    "gpu_count": int(count),
                    "price_hourly_usd_instance": round(instance_price, 4),
                    "price_hourly_usd_per_gpu": round(per_gpu, 4),
                    "price_reserved_usd": None,
                    "reserved_duration": None,
                    "timestamp": timestamp(),
                })

    df = pd.DataFrame(rows)
    if not df.empty:
        df = (df.sort_values(["gpu_model","price_hourly_usd_instance"])
                .drop_duplicates(subset=["gpu_model","instance_type","price_hourly_usd_instance"], keep="last")
                .reset_index(drop=True))
    return df

# Example
df_ovh = scrape_ovhcloud_correct()
print(df_ovh.head(20))

   provider  region gpu_model       instance_type  gpu_count  \
0  OVHcloud  Global      H100  public-cloud/Price          2   
1  OVHcloud  Global      H100  public-cloud/Price          4   

   price_hourly_usd_instance  price_hourly_usd_per_gpu price_reserved_usd  \
0                       5.98                    2.9900               None   
1                      11.97                    2.9925               None   

  reserved_duration                   timestamp  
0              None  2025-09-10T12:22:33.642693  
1              None  2025-09-10T12:22:33.642767  


In [129]:
# === ONE-CELL: unify → baselines (On-Demand) → score → ROI → Silicon-style prediction ===
# Bullet-proof: guarantees per-GPU $/hr column and avoids KeyErrors. Py3.8 friendly.

import re, math, json, numpy as np, pandas as pd
from pathlib import Path
from datetime import datetime, timezone

# ---------- config ----------
BASE = Path("docs/data")
LATEST_DIR = BASE / "latest"
DERIVED_DIR = BASE / "derived"
for d in (LATEST_DIR, DERIVED_DIR):
    d.mkdir(parents=True, exist_ok=True)

def _now_iso():
    return datetime.now(timezone.utc).isoformat(timespec="seconds")

PRICE_COL = "effective_price_usd_per_gpu_hr"  # single source of truth

# providers to exclude from *all* outputs
PROVIDERS_EXCLUDE = ["CoreWeave"]
_excl_set = {p.casefold() for p in PROVIDERS_EXCLUDE}

def _exclude_providers(df: pd.DataFrame) -> pd.DataFrame:
    if "provider" not in df.columns:
        return df
    prov_norm = df["provider"].astype(str).str.strip().str.casefold()
    return df[~prov_norm.isin(_excl_set)].copy()

# ---------- helpers ----------
def to_hours(s):
    if not isinstance(s,str) or not s.strip(): return np.nan
    s = s.strip().lower()
    m = re.match(r"^(\d+(?:\.\d+)?)\s*([a-z]+)s?$", s)
    if not m: return np.nan
    q, u = float(m.group(1)), m.group(2)
    mult = {"h":1,"hr":1,"hour":1,"d":24,"day":24,"w":168,"wk":168,"week":168,"mo":720,"month":720}.get(u, np.nan)
    return q*mult

def _numify(series):
    # handles None/'None'/'$1,234'/ '1.2/hr' etc.
    return pd.to_numeric(series.astype(str).str.replace(r"[^\d\.\-]", "", regex=True), errors="coerce")

EXPECTED = {
    "provider": ["provider","Provider","name"],
    "region": ["region","Region","location","geo","cloud_region","area"],
    "gpu_model": ["gpu_model","gpu","model","GPU"],
    "gpu_count": ["gpu_count","count","gpus","GPU_count"],
    "type": ["type","price_type"],
    "duration": ["duration","reserved_duration","term"],
    "price_hourly_usd": ["price_hourly_usd","price","usd_per_gpu_hr","price_hourly_usd_per_gpu"],
    "price_reserved_usd": ["price_reserved_usd","reserved_usd_per_gpu_hr","reserved_price"],
    "price_hourly_usd_instance": ["price_hourly_usd_instance","instance_price","node_hourly_usd"],
    "timestamp": ["timestamp","fetched_at_utc","ts_utc","ts_iso","time","start_time_iso"],
    "source_url": ["source_url","url","link"],
    # optional specs (kept, but NOT used in prediction right now)
    "gpu_vram_gb": ["gpu_vram_gb","gpu_ram_gb","vram_gb"],
    "cpu_cores": ["cpu_cores","vcpus","cpu","cpu_count"],
    "cpu_ram_gb": ["cpu_ram_gb","memory_gb","mem_gb","ram_gb"],
}

def _map_expected(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for tgt, cands in EXPECTED.items():
        if tgt not in out.columns:
            for c in cands:
                if c in out.columns:
                    out[tgt] = out[c]; break
        if tgt not in out.columns:
            out[tgt] = pd.NA
    return out

def _derive_effective_price(df: pd.DataFrame) -> pd.Series:
    """Always returns float Series for per-GPU $/hr, derived from any price fields + duration."""
    n = len(df)
    eff = pd.Series(np.nan, index=df.index, dtype="float64")
    # direct per-GPU if present
    if PRICE_COL in df.columns:
        eff = pd.to_numeric(df[PRICE_COL], errors="coerce")
    # explicit per-GPU hourly
    if "price_hourly_usd" in df.columns:
        ph = _numify(df["price_hourly_usd"])
        eff = eff.fillna(ph)
    # split instance hourly by gpu_count
    if "price_hourly_usd_instance" in df.columns:
        inst = _numify(df["price_hourly_usd_instance"])
        g = pd.to_numeric(df.get("gpu_count", pd.Series([np.nan]*n)), errors="coerce")
        m = eff.isna() & inst.notna() & g.notna() & (g > 0)
        eff.loc[m] = inst.loc[m] / g.loc[m]
    # reserved: detect total-for-term vs hourly
    if "price_reserved_usd" in df.columns:
        res = _numify(df["price_reserved_usd"])
        dur_h = df.get("duration", pd.Series([""]*n)).apply(to_hours)
        m_tot = eff.isna() & res.notna() & dur_h.notna() & res.gt(10)
        eff.loc[m_tot] = res.loc[m_tot] / dur_h.loc[m_tot]
        m_hr = eff.isna() & res.notna() & ~m_tot
        eff.loc[m_hr] = res.loc[m_hr]
    return pd.to_numeric(eff, errors="coerce")

def normalise_any(df: pd.DataFrame) -> pd.DataFrame:
    """Canonicalize cols + compute per-GPU $/hr. Keeps only sane H100/H200 rows."""
    df = _map_expected(df)
    # types & defaults
    df["gpu_model"] = df["gpu_model"].astype(str).str.upper().str.strip()
    df["region"] = df["region"].astype(str).replace({"nan":None}).fillna("Global").str.strip()
    df["type"] = df["type"].astype(str).replace({"nan":None}).fillna("On-Demand").str.strip()
    df["duration"] = df["duration"].astype(str).replace({"nan":None,"":None}).fillna("1h").str.strip()
    df["gpu_count"] = pd.to_numeric(df["gpu_count"], errors="coerce")
    df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce", utc=True)
    # derive per-GPU $/hr
    df[PRICE_COL] = _derive_effective_price(df)
    # bounds + models
    df = df[df["gpu_model"].isin(["H100","H200"])]
    df = df[df[PRICE_COL].between(0.05, 200)].copy()
    # final ISO timestamp
    df["timestamp"] = df["timestamp"].dt.strftime("%Y-%m-%dT%H:%M:%SZ")
    keep = ["provider","region","gpu_model","type","duration","gpu_count",
            "price_hourly_usd","price_reserved_usd","price_hourly_usd_instance",
            PRICE_COL,"timestamp","source_url","gpu_vram_gb","cpu_cores","cpu_ram_gb"]
    extra = [c for c in df.columns if c not in keep]
    return df[keep + extra]

def ensure_price_col(df: pd.DataFrame) -> pd.DataFrame:
    """Guarantee a numeric per-GPU $/hr column named PRICE_COL."""
    out = df.copy()
    # exact column
    if PRICE_COL in out.columns:
        out[PRICE_COL] = pd.to_numeric(out[PRICE_COL], errors="coerce")
    # similar-name rescue
    if PRICE_COL not in out.columns or out[PRICE_COL].isna().all():
        cand = None
        for c in out.columns:
            if re.search(r"effective.*gpu.*(hr|hour)", c, flags=re.I):
                cand = c; break
        if cand and cand != PRICE_COL:
            out = out.rename(columns={cand: PRICE_COL})
            out[PRICE_COL] = pd.to_numeric(out[PRICE_COL], errors="coerce")
    # last resort: derive
    if PRICE_COL not in out.columns or out[PRICE_COL].isna().all():
        out[PRICE_COL] = _derive_effective_price(out)
    out[PRICE_COL] = pd.to_numeric(out[PRICE_COL], errors="coerce")
    return out

def percentile_rev(s: pd.Series) -> pd.Series:
    r = s.rank(pct=True, method="average")
    return 1.0 - (r - r.min())/(r.max()-r.min()+1e-12)

def add_priceiq_score(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    grp = ["gpu_model","region"] if df["region"].nunique() > 1 else ["gpu_model"]
    df["_pct"] = df.groupby(grp)[PRICE_COL].transform(percentile_rev)
    cap = np.log1p(df["gpu_count"].fillna(1))/np.log1p(256)  # 0..1
    df["_cap"] = 15.0*cap.clip(0,1)  # up to +15
    def rec(ts):
        try:
            t = pd.to_datetime(ts, utc=True)
            age_h = max(0.0, (pd.Timestamp.utcnow()-t).total_seconds()/3600)
            return 10.0*math.exp(-age_h/72.0)  # up to +10
        except:
            return 0.0
    df["_rec"] = df["timestamp"].map(rec)
    df["priceiq_score"] = (df["_pct"]*75.0 + df["_cap"] + df["_rec"]).clip(0,100)
    return df.drop(columns=["_pct","_cap","_rec"], errors="ignore")

# ---------- gather frames ----------
frames = []

# iterate over a *dict* (copy) so .items() exists; safe if globals change during loop
for name, obj in globals().copy().items():
    if isinstance(obj, pd.DataFrame) and (name.startswith(("df_","latest_"))) and not obj.empty:
        frames.append(obj.copy())

if not frames:
    for p in sorted(LATEST_DIR.glob("*_latest.csv")):
        try:
            frames.append(pd.read_csv(p, low_memory=False))
        except Exception as e:
            print(f"[WARN] failed to load {p.name}: {e}")

if not frames:
    raise RuntimeError("No provider frames found (in-memory or docs/data/latest/*). Run scrapers first.")


# -------- EXCLUSION: drop CoreWeave from everything --------
norm = _exclude_providers(norm)

norm = ensure_price_col(norm)
norm = norm[norm[PRICE_COL].between(0.05, 200)]
if norm.empty:
    raise RuntimeError("No usable per-GPU prices after normalization.")

# de-dupe newest by key
key = ["provider","region","gpu_model","type","duration","gpu_count",PRICE_COL]
norm = (norm.sort_values("timestamp")
            .drop_duplicates(subset=key, keep="last")
            .reset_index(drop=True))

# ---------- 1) Market baselines (On-Demand only; flat columns) ----------
gcols = ["gpu_model", "region"]  # pool for baselines
od = norm[norm["type"].str.contains("On-Demand", case=False, na=False) & norm[PRICE_COL].notna()].copy()
if od.empty:
    raise RuntimeError("No On-Demand rows available to build market baselines.")

# build market stats with named aggregations (flat columns in all pandas versions)
market = (
    od.groupby(gcols, as_index=False)
      .agg(
          market_count =(PRICE_COL, 'size'),
          market_mean  =(PRICE_COL, 'mean'),
          market_median=(PRICE_COL, 'median'),
          market_p10   =(PRICE_COL, lambda s: s.quantile(0.10)),
          market_p25   =(PRICE_COL, lambda s: s.quantile(0.25)),
          market_p75   =(PRICE_COL, lambda s: s.quantile(0.75)),
          market_p90   =(PRICE_COL, lambda s: s.quantile(0.90)),
      )
)

# merge onto the full set
scored = norm.merge(market, on=gcols, how="left").copy()

# NEW GUARD: ensure market_* columns exist even if merge produced none (edge cases)
for _c in ["market_count","market_mean","market_median","market_p25","market_p75","market_p10","market_p90"]:
    if _c not in scored.columns:
        scored[_c] = np.nan

# if any older pandas produced MultiIndex columns for any reason, flatten as a safeguard
if any(isinstance(c, tuple) for c in scored.columns):
    scored.columns = ["_".join(str(x) for x in c if x) if isinstance(c, tuple) else c]

# guarantee the price column exists post-merge
scored = ensure_price_col(scored)

# --- premium vs market median (robust Series, safe zero/NaN guard) ---
if "market_median" in scored.columns:
    mm = pd.to_numeric(scored["market_median"], errors="coerce")
else:
    mm = pd.Series(np.nan, index=scored.index, dtype="float64")
mm = mm.mask(mm == 0)  # treat zeros as missing to avoid div/0
scored["premium_vs_median"] = (scored[PRICE_COL] - mm) / mm
scored["asof_utc"] = _now_iso()

# ---- fill any missing market stats with model-wide fallbacks, then self-price as last resort
missing = scored["market_median"].isna() if "market_median" in scored.columns else pd.Series(True, index=scored.index)
if missing.any():
    fb = (
        od.groupby(["gpu_model"])[PRICE_COL]
          .agg(
              market_count_fb="size",
              market_mean_fb ="mean",
              market_median_fb="median",
              market_p25_fb  =lambda s: s.quantile(0.25),
              market_p75_fb  =lambda s: s.quantile(0.75),
              market_p10_fb  =lambda s: s.quantile(0.10),
              market_p90_fb  =lambda s: s.quantile(0.90),
          )
          .reset_index()
    )
    scored = scored.merge(fb, on="gpu_model", how="left")
    for col in ["market_count","market_mean","market_median","market_p25","market_p75","market_p10","market_p90"]:
        if col in scored.columns and f"{col}_fb" in scored.columns:
            scored[col] = scored[col].where(scored[col].notna(), scored[f"{col}_fb"])
    scored.drop(columns=[c for c in scored.columns if c.endswith("_fb")], inplace=True, errors="ignore")
    # last resort: if still NaN, copy row price into market columns
    for col in ["market_median","market_mean","market_p25","market_p75","market_p10","market_p90"]:
        if col in scored.columns:
            scored[col] = scored[col].fillna(scored[PRICE_COL])

    # NEW SAFE RECOMPUTE: robust again even if column missing
    mm = pd.to_numeric(scored.get("market_median", pd.Series(np.nan, index=scored.index)), errors="coerce")
    mm = mm.mask(mm == 0)
    scored["premium_vs_median"] = (scored[PRICE_COL] - mm) / mm

# ---------- 2) Price-IQ score & export ----------
scored = add_priceiq_score(scored)
scores_path = DERIVED_DIR / "provider_scores_latest.csv"
scored.to_csv(scores_path, index=False)
print("✔ wrote", scores_path, f"({len(scored)} rows)")

# ---------- 3) ROI (cheapest total cost by scenario, On-Demand pool) ----------
def roi_table(df: pd.DataFrame) -> pd.DataFrame:
    base = df[df[PRICE_COL].notna()].copy()
    if base.empty: return pd.DataFrame()
    scenarios, models = [], [m for m in ["H100","H200"] if m in set(base["gpu_model"])] or sorted(base["gpu_model"].unique())
    for m in models:
        for c in [8,16,32,64]:
            for d in ["1 hour","1 day","1 week","1 month"]:
                h = to_hours(d)
                if not h or np.isnan(h): continue
                sub = base[base["gpu_model"].eq(m)].copy()
                if sub.empty: continue
                sub["total_cost"] = sub[PRICE_COL] * c * h
                r = sub.nsmallest(1, ["total_cost", PRICE_COL])
                if r.empty: continue
                r = r.iloc[0]
                scenarios.append({
                    "gpu_model": m,
                    "gpu_count": int(c),
                    "duration": d,
                    "best_provider": r["provider"],
                    "best_region": r["region"],
                    "price_per_gpu_hr": round(float(r[PRICE_COL]), 4),
                    "total_cost_usd": round(float(r["total_cost"]), 2),
                    "timestamp": r["timestamp"]
                })
    return pd.DataFrame(scenarios)

roi_df = roi_table(od)
roi_path = DERIVED_DIR / "roi_comparison.csv"
roi_df.to_csv(roi_path, index=False)
print("✔ wrote", roi_path, f"({len(roi_df)} rows)")

# ---------- 4) Silicon-style prediction (GPU + type + region; CPU/RAM = N/A for now) ----------
REQ = dict(
    gpu_model="H100",          # "H100" or "H200"
    type="On-Demand",          # predict from On-Demand pool
    gpu_vram_gb=None,          # N/A
    cpu_cores=None,            # N/A
    cpu_ram_gb=None,           # N/A
    region="US"                # prefer exact region; fallback Global → mixed
)

def predict_price_quantiles(data: pd.DataFrame, req: dict):
    df = data[(data["gpu_model"].str.upper()==str(req.get("gpu_model","")).upper())].copy()
    if "type" in req and isinstance(req["type"], str) and req["type"].strip():
        df = df[df["type"].str.contains(req["type"], case=False, na=False)]
    if df.empty:
        return {"n":0, "used_region":"none", "p10":None,"p25":None,"p50":None,"p75":None,"p90":None}
    region = str(req.get("region","Global"))
    df_reg = df[df["region"]==region]
    used_reg = region if not df_reg.empty else ("Global" if not df[df["region"].str.lower().eq("global")].empty else "mixed")
    if df_reg.empty:
        df_reg = df[df["region"].str.lower().eq("global")] if used_reg=="Global" else df
    # equal weights; (optional) enable recency downweighting
    # ages_h = (pd.Timestamp.utcnow() - pd.to_datetime(df_reg["timestamp"], utc=True)).dt.total_seconds()/3600
    # w = np.exp(-np.clip(ages_h,0,None)/72.0)
    w = np.ones(len(df_reg), dtype=float)
    vals = df_reg[PRICE_COL].astype(float).values
    order = np.argsort(vals); v, ww = vals[order], w[order]
    c = np.cumsum(ww); total = c[-1]
    def q(p): return None if total<=0 else round(float(np.interp(p*total, c, v)), 4)
    return {"n": int(len(df_reg)), "used_region": used_reg,
            "p10": q(0.10), "p25": q(0.25), "p50": q(0.50), "p75": q(0.75), "p90": q(0.90)}

pred = predict_price_quantiles(od, REQ)
pred_record = {"asof_utc": _now_iso(), **REQ, **pred}
(DERIVED_DIR / "price_predict_snapshot.json").write_text(json.dumps(pred_record, indent=2))
print("✔ wrote", DERIVED_DIR / "price_predict_snapshot.json")
print(json.dumps(pred_record, indent=2))

# ---------- 5) Export market index ----------
market_path = DERIVED_DIR / "market_index.csv"
market.to_csv(market_path, index=False)
print("✔ wrote", market_path)

# Keep notebook output light
_preview_cols = [c for c in ["provider","region","gpu_model","type",PRICE_COL,"market_median","priceiq_score"] if c in scored.columns]
print("\nPreview — provider_scores_latest.csv:")
print(scored[_preview_cols].head(10).to_string(index=False))

print("\nPreview — roi_comparison.csv:")
print(roi_df.head(6).to_string(index=False))


✔ wrote docs/data/derived/provider_scores_latest.csv (24 rows)
✔ wrote docs/data/derived/roi_comparison.csv (32 rows)
✔ wrote docs/data/derived/price_predict_snapshot.json
{
  "asof_utc": "2025-09-10T15:28:44+00:00",
  "gpu_model": "H100",
  "type": "On-Demand",
  "gpu_vram_gb": null,
  "cpu_cores": null,
  "cpu_ram_gb": null,
  "region": "US",
  "n": 1,
  "used_region": "US",
  "p10": 1.99,
  "p25": 1.99,
  "p50": 1.99,
  "p75": 1.99,
  "p90": 1.99
}
✔ wrote docs/data/derived/market_index.csv

Preview — provider_scores_latest.csv:
    provider  region gpu_model       type  effective_price_usd_per_gpu_hr  market_median  priceiq_score
    OVHcloud  Global      H100  On-Demand                          2.9900           2.25      31.297884
    OVHcloud  Global      H100  On-Demand                          2.9925           2.25      26.428724
      Nebius      US      H200  On-Demand                          2.3000           2.30      11.873683
   Shadeform  Global      H200  On-Demand     

In [130]:
# === ROI CALCULATORS: tenant quotes + provider earnings =======================
# Outputs:
#   - docs/data/derived/tenant_roi_quotes.csv
#   - docs/data/derived/provider_roi_projection.csv
#   - docs/data/derived/provider_roi_summary.json
#
# Relies on provider prices in memory as `scored` OR docs/data/derived/provider_scores_latest.csv

import json, math, numpy as np, pandas as pd
from dataclasses import dataclass
from pathlib import Path
from typing import Tuple, Dict
from datetime import datetime, timezone

# ---------------- basics / IO ----------------
BASE = Path("docs/data")
DERIVED = BASE / "derived"
DERIVED.mkdir(parents=True, exist_ok=True)

def _now_iso(): return datetime.now(timezone.utc).isoformat(timespec="seconds")

def _safe_float(x, default=np.nan):
    try:
        v = float(x)
        if np.isfinite(v): return v
    except Exception:
        pass
    return default

def to_hours(s):
    if not isinstance(s,str) or not s.strip(): return np.nan
    t = s.strip().lower().rstrip("s")
    m = __import__("re").match(r"^(\d+(?:\.\d+)?)\s*([a-z]+)$", t)
    if not m: return np.nan
    q,u = float(m.group(1)), m.group(2)
    mult = {"h":1,"hr":1,"hour":1,"d":24,"day":24,"w":168,"wk":168,"week":168,"mo":720,"month":720}.get(u, np.nan)
    return q*mult

# Ensure we have prices in a DataFrame named `scored`
try:
    assert isinstance(scored, pd.DataFrame) and not scored.empty
except Exception:
    prices_path = DERIVED / "provider_scores_latest.csv"
    scored = pd.read_csv(prices_path, low_memory=False) if prices_path.exists() else pd.DataFrame()
if scored.empty:
    raise RuntimeError("No price table available (scored). Run the pricing pipeline first.")

# Make sure price column exists
PRICE_COL = "effective_price_usd_per_gpu_hr"
if PRICE_COL not in scored.columns:
    raise RuntimeError(f"Missing {PRICE_COL} in provider prices.")

# ---------------- A) TENANT ROI / COST QUOTES ----------------
@dataclass
class UsageInputs:
    gpu_model: str       # "H100" | "H200"
    region: str          # "US" | "EU" | "Global" ...
    count: int           # number of GPUs
    duration: str        # "1 hour" | "1 day" | "1 week" | "1 month"
    utilization_pct: float = 100.0     # % of the time they’re busy
    type: str = "On-Demand"
    provider: str = "All providers"    # specific provider or "All providers"
    use_predictions: bool = False      # if True, use group p50 price for cost

# Helper: group median (p50) for a request (by gpu_model + region, falling back)
def _group_p50(df: pd.DataFrame, gpu_model: str, region: str) -> float:
    d = df[df["gpu_model"].str.upper()==gpu_model.upper()]
    if region and isinstance(region,str):
        d_reg = d[d["region"]==region]
        if d_reg.empty:
            d_reg = d[d["region"].str.lower().eq("global")]
        if d_reg.empty:
            d_reg = d
    else:
        d_reg = d
    return float(d_reg[PRICE_COL].median()) if not d_reg.empty else np.nan

def build_usage_quotes(inputs: UsageInputs, price_df: pd.DataFrame) -> pd.DataFrame:
    # filter rows
    df = price_df.copy()
    df = df[df["gpu_model"].str.upper()==inputs.gpu_model.upper()]
    df = df[df["type"].str.contains(inputs.type, case=False, na=False)]
    # region preference: exact -> Global -> any
    df_reg = df[df["region"]==inputs.region]
    used_region = inputs.region
    if df_reg.empty:
        df_reg = df[df["region"].str.lower().eq("global")]
        used_region = "Global"
    if df_reg.empty:
        df_reg = df
        used_region = "mixed"

    if inputs.provider and inputs.provider != "All providers":
        df_reg = df_reg[df_reg["provider"].str.strip().str.lower()==inputs.provider.strip().lower()]

    if df_reg.empty:
        return pd.DataFrame(columns=[
            "provider","region","gpu_model","type","duration","gpu_count","utilization_pct",
            "price_per_gpu_hr","total_cost_usd","price_source","source_url","asof_utc"
        ])

    hrs  = to_hours(inputs.duration)
    util = max(0.0, min(100.0, float(inputs.utilization_pct))) / 100.0
    cnt  = int(inputs.count)

    # price selection
    if inputs.use_predictions:
        p50 = _group_p50(df_reg, inputs.gpu_model, used_region)
        price_col = pd.Series([p50] * len(df_reg), index=df_reg.index)
        src = "pred_p50"
    else:
        price_col = df_reg[PRICE_COL]
        src = "observed"

    out = df_reg.copy()
    out["gpu_count"]       = cnt
    out["duration"]        = inputs.duration
    out["utilization_pct"] = inputs.utilization_pct
    out["price_per_gpu_hr"]= price_col.astype(float)
    out["total_cost_usd"]  = (out["price_per_gpu_hr"] * cnt * hrs * util).round(2)
    out["price_source"]    = src
    out["asof_utc"]        = _now_iso()

    keep = ["provider","region","gpu_model","type","duration","gpu_count","utilization_pct",
            "price_per_gpu_hr","total_cost_usd","price_source","source_url","asof_utc"]
    # sort by total cost ascending
    return out[keep].sort_values(["total_cost_usd","price_per_gpu_hr","provider"]).reset_index(drop=True)

# Example inputs (hook your UI to these)
usage = UsageInputs(
    gpu_model="H100",
    region="US",
    count=8,
    duration="1 week",
    utilization_pct=100,
    type="On-Demand",
    provider="All providers",
    use_predictions=False,      # flip to True if you want p50-based “market” quote
)

tenant_quotes = build_usage_quotes(usage, scored)
tenant_path = DERIVED / "tenant_roi_quotes.csv"
tenant_quotes.to_csv(tenant_path, index=False)
print(f"✔ wrote {tenant_path} ({len(tenant_quotes)} rows)")
display(tenant_quotes.head(10))

# ---------------- B) PROVIDER ROI / EARNINGS PROJECTION ----------------
@dataclass
class ROIInputs:
    gpu_model: str = "H100"
    region: str = "Global"
    fleet_gpus: int = 80                 # physical GPUs
    price_per_gpu_hr_initial: float = 1.25
    oversub_multiplier: float = 2.0      # effective util = min(util * oversub, 0.99)
    price_erosion_yoy: float = 0.20      # 20% YoY decline
    util_model: str = "linear"           # "linear" or "logistic"
    util_start_pct: float = 0.35
    util_peak_pct: float = 0.85
    util_months_to_peak: int = 18
    watts_per_gpu: float = 350.0
    usd_per_kwh: float = 0.12
    other_opex_usd_per_month: float = 0.0
    capex_usd: float = 2_400_000.0
    capex_month0: bool = True
    depreciation_months: int = 36
    tax_rate: float = 0.0                # simple EBIT tax
    months: int = 60                     # projection horizon

def _price_erosion_path(p0: float, months: int, yoy: float) -> np.ndarray:
    # Convert YoY decline to monthly factor
    m = 1.0 - (1.0 - yoy) ** (1.0 / 12.0)
    return p0 * (1.0 - m) ** np.arange(months)

def _util_path_linear(start: float, peak: float, months_to_peak: int, n: int) -> np.ndarray:
    x = np.linspace(0, 1, max(2, months_to_peak))
    path = np.interp(np.arange(n), np.arange(months_to_peak), start + (peak - start) * x)
    path = np.clip(path, min(start,peak), max(start,peak))
    if n > months_to_peak:
        tail = np.full(n - months_to_peak, peak)
        path = np.concatenate([path[:months_to_peak], tail])
    return path

def _util_path_logistic(start: float, peak: float, months_to_peak: int, n: int) -> np.ndarray:
    # Simple logistic reaching ~peak at months_to_peak
    t = np.arange(n)
    L = peak
    k = 6.0 / max(1, months_to_peak)      # controls steepness
    x0 = months_to_peak / 2.0
    y = L / (1.0 + np.exp(-k * (t - x0)))
    # shift so y(0)=start approximately
    adj = (y[0] if n>0 else 0.0)
    if adj != 0:
        y = (y - adj) * ((peak - start) / (y[-1] - adj + 1e-9)) + start
    return np.clip(y, 0.0, peak)

def _irr(cashflows: np.ndarray, guess: float = 0.02, max_iter: int = 100, tol: float = 1e-7) -> float:
    # Secant method IRR to avoid numpy_financial dependency
    def npv(rate):
        return np.sum(cashflows / (1.0 + rate) ** np.arange(len(cashflows)))
    r0, r1 = guess, guess * 1.1 + 1e-6
    f0, f1 = npv(r0), npv(r1)
    for _ in range(max_iter):
        if abs(f1 - f0) < 1e-12: break
        r2 = r1 - f1 * (r1 - r0) / (f1 - f0)
        if not np.isfinite(r2) or r2 <= -0.9999: r2 = 0.0
        f2 = npv(r2)
        if abs(f2) < tol: return r2
        r0, f0, r1, f1 = r1, f1, r2, f2
    return r1

def project_provider(inputs: ROIInputs) -> Tuple[pd.DataFrame, Dict]:
    n = int(inputs.months)
    # price and utilization paths
    price = _price_erosion_path(inputs.price_per_gpu_hr_initial, n, inputs.price_erosion_yoy)
    if inputs.util_model.lower().startswith("log"):
        util = _util_path_logistic(inputs.util_start_pct, inputs.util_peak_pct,
                                   max(1, inputs.util_months_to_peak), n)
    else:
        util = _util_path_linear(inputs.util_start_pct, inputs.util_peak_pct,
                                 max(1, inputs.util_months_to_peak), n)
    util = np.clip(util, 0.0, 0.999)

    # ✅ FIX: provide both a_min and a_max
    delivered_util = np.clip(util * max(0.0, float(inputs.oversub_multiplier)), 0.0, 0.999)

    # monthly hours
    H = 720.0  # 30 days
    # revenues
    gpu_hours = inputs.fleet_gpus * delivered_util * H
    revenue  = gpu_hours * price

    # power costs (scale with delivered utilization)
    kwh_per_gpu_month = (inputs.watts_per_gpu / 1000.0) * 24.0 * 30.0 * delivered_util
    power_cost = inputs.fleet_gpus * kwh_per_gpu_month * inputs.usd_per_kwh

    other_opex = np.full(n, float(inputs.other_opex_usd_per_month))
    depreciation = np.zeros(n)
    if inputs.depreciation_months > 0:
        dep = inputs.capex_usd / inputs.depreciation_months
        depreciation[:inputs.depreciation_months] = dep

    ebitda = revenue - power_cost - other_opex
    ebit   = ebitda - depreciation
    tax    = np.where(ebit > 0, inputs.tax_rate * ebit, 0.0)
    net_income = ebit - tax

    # cash flow: EBITDA - tax - initial capex (if month0)
    cashflow = ebitda - tax
    if inputs.capex_month0 and inputs.capex_usd > 0:
        cashflow[0] -= inputs.capex_usd
    cum_cash = np.cumsum(cashflow)

    # KPIs
    payback_month = int(np.argmax(cum_cash >= 0)) if np.any(cum_cash >= 0) else None
    irr_m = _irr(cashflow)  # monthly IRR
    irr_ann = (1.0 + irr_m) ** 12 - 1.0 if np.isfinite(irr_m) else np.nan

    months = np.arange(1, n+1)
    df = pd.DataFrame({
        "month": months,
        "price_per_gpu_hr": price.round(6),
        "util_pct": (util*100).round(2),
        "delivered_util_pct": (delivered_util*100).round(2),
        "revenue_usd": revenue.round(2),
        "power_cost_usd": power_cost.round(2),
        "other_opex_usd": other_opex.round(2),
        "depreciation_usd": depreciation.round(2),
        "ebitda_usd": ebitda.round(2),
        "ebit_usd": ebit.round(2),
        "tax_usd": tax.round(2),
        "net_income_usd": net_income.round(2),
        "cashflow_usd": cashflow.round(2),
        "cum_cash_usd": cum_cash.round(2),
    })

    summary = {
        "gpu_model": inputs.gpu_model,
        "region": inputs.region,
        "fleet_gpus": inputs.fleet_gpus,
        "price_per_gpu_hr_initial": inputs.price_per_gpu_hr_initial,
        "oversub_multiplier": inputs.oversub_multiplier,
        "price_erosion_yoy": inputs.price_erosion_yoy,
        "util_model": inputs.util_model,
        "util_start_pct": inputs.util_start_pct,
        "util_peak_pct": inputs.util_peak_pct,
        "util_months_to_peak": inputs.util_months_to_peak,
        "watts_per_gpu": inputs.watts_per_gpu,
        "usd_per_kwh": inputs.usd_per_kwh,
        "other_opex_usd_per_month": inputs.other_opex_usd_per_month,
        "capex_usd": inputs.capex_usd,
        "capex_month0": inputs.capex_month0,
        "depreciation_months": inputs.depreciation_months,
        "tax_rate": inputs.tax_rate,
        "months": n,
        "payback_month": payback_month,
        "irr_monthly": float(irr_m) if np.isfinite(irr_m) else None,
        "irr_annualized": float(irr_ann) if np.isfinite(irr_ann) else None,
    }
    return df, summary


# Example provider ROI inputs (wire these to your dashboard controls)
provider_inputs = ROIInputs(
    gpu_model="H100",
    region="Global",
    fleet_gpus=80,
    price_per_gpu_hr_initial= _safe_float(scored[scored["gpu_model"]=="H100"][PRICE_COL].median(), 1.25),
    oversub_multiplier=2.0,
    price_erosion_yoy=0.20,
    util_model="linear",
    util_start_pct=0.35,
    util_peak_pct=0.85,
    util_months_to_peak=18,
    watts_per_gpu=350,
    usd_per_kwh=0.12,
    other_opex_usd_per_month=0.0,
    capex_usd=2_400_000,
    capex_month0=True,
    depreciation_months=36,
    tax_rate=0.0,
    months=60,
)

provider_monthly, provider_summary = project_provider(provider_inputs)
prov_path = DERIVED / "provider_roi_projection.csv"
provider_monthly.to_csv(prov_path, index=False)
(DERIVED / "provider_roi_summary.json").write_text(json.dumps(provider_summary, indent=2))
print(f"✔ wrote {prov_path} ({len(provider_monthly)} rows)")
print(f"✔ wrote {DERIVED / 'provider_roi_summary.json'}")
display(provider_monthly.head(6))
print("Summary:", json.dumps(provider_summary, indent=2))


✔ wrote docs/data/derived/tenant_roi_quotes.csv (1 rows)


,provider,region,gpu_model,type,duration,gpu_count,utilization_pct,price_per_gpu_hr,total_cost_usd,price_source,source_url,asof_utc
0,VoltagePark,US,H100,On-Demand,1 week,8,100,1.99,2674.56,observed,https://dashboard.voltagepark.com/order/config...,2025-09-10T15:28:55+00:00


✔ wrote docs/data/derived/provider_roi_projection.csv (60 rows)
✔ wrote docs/data/derived/provider_roi_summary.json


,month,price_per_gpu_hr,util_pct,delivered_util_pct,revenue_usd,power_cost_usd,other_opex_usd,depreciation_usd,ebitda_usd,ebit_usd,tax_usd,net_income_usd,cashflow_usd,cum_cash_usd
0,1,2.325000,35.00,70.00,93744.00,1693.44,0.0,66666.67,92050.56,25383.89,0.0,25383.89,-2307949.44,-2307949.44
1,2,2.282165,37.94,75.88,99749.42,1835.75,0.0,66666.67,97913.68,31247.01,0.0,31247.01,97913.68,-2210035.76
2,3,2.240120,40.88,81.76,105501.75,1978.05,0.0,66666.67,103523.70,36857.03,0.0,36857.03,103523.70,-2106512.07
3,4,2.198849,43.82,87.65,111008.26,2120.36,0.0,66666.67,108887.90,42221.23,0.0,42221.23,108887.90,-1997624.17
4,5,2.158339,46.76,93.53,116276.06,2262.66,0.0,66666.67,114013.40,47346.73,0.0,47346.73,114013.40,-1883610.77
5,6,2.118575,49.71,99.41,121312.08,2404.97,0.0,66666.67,118907.11,52240.44,0.0,52240.44,118907.11,-1764703.66


Summary: {
  "gpu_model": "H100",
  "region": "Global",
  "fleet_gpus": 80,
  "price_per_gpu_hr_initial": 2.325,
  "oversub_multiplier": 2.0,
  "price_erosion_yoy": 0.2,
  "util_model": "linear",
  "util_start_pct": 0.35,
  "util_peak_pct": 0.85,
  "util_months_to_peak": 18,
  "watts_per_gpu": 350,
  "usd_per_kwh": 0.12,
  "other_opex_usd_per_month": 0.0,
  "capex_usd": 2400000,
  "capex_month0": true,
  "depreciation_months": 36,
  "tax_rate": 0.0,
  "months": 60,
  "payback_month": 23,
  "irr_monthly": 0.03261738789953898,
  "irr_annualized": 0.4698506356943497
}


In [131]:
# === Compute calculator snapshot: bands per provider_type & GPU model ===
import numpy as np, pandas as pd, math, re
from pathlib import Path

DERIVED = Path("docs/data/derived")
DERIVED.mkdir(parents=True, exist_ok=True)

# ---- provider typing (same mapping you used elsewhere) ----
_HYPERSCALER = {"AWS","Amazon","GCP","Google Cloud","Azure","Microsoft Azure","OCI","Oracle Cloud"}
_BROKER      = {"Vast.ai","Hydra Host (Brokkr)","Brokkr","TensorDock","Shadeform"}
_NEO         = {"VoltagePark","CoreWeave","Lambda Labs","RunPod","Runpod","Paperspace","Nebius","SFCompute","OVHcloud"}
OVERRIDES    = {
    # "RunPod": "Broker",    # uncomment to re-bucket RunPod
    # "Runpod": "Broker",
}

def infer_provider_type(name: str) -> str:
    n = str(name or "").strip()
    if n in OVERRIDES: return OVERRIDES[n]
    if n in _HYPERSCALER: return "Hyperscaler"
    if n in _BROKER:      return "Broker"
    if n in _NEO:         return "Neocloud"
    u = n.lower()
    if any(k in u for k in ["aws","amazon","gcp","google","azure","oci","oracle"]): return "Hyperscaler"
    if any(k in u for k in ["vast","brokkr","hydra","tensordock","shadeform"]):     return "Broker"
    return "Neocloud"

# ---- term hours helper ----
def hours_for_term(term: str) -> int:
    t = str(term).strip().lower()
    if t in {"1 hour","1h","hour"}:   return 1
    if t in {"1 day","1d","day"}:     return 24
    if t in {"1 week","1w","week"}:   return 7*24
    if t in {"1 month","1mo","month"}:return 30*24
    # “6 months”, “3 months”, etc.
    m = re.match(r"(\d+)\s*month", t)
    if m: return int(m.group(1))*30*24
    raise ValueError(f"Unrecognised term: {term}")

# ---- weighted quantiles for bands ----
def weighted_quantiles(vals, weights, qs=(0.10,0.50,0.90)):
    v = np.asarray(vals, dtype=float)
    w = np.asarray(weights, dtype=float)
    m = np.isfinite(v) & np.isfinite(w) & (w>=0)
    v, w = v[m], w[m]
    if v.size == 0 or w.sum()==0:
        return {q: np.nan for q in qs}
    order = np.argsort(v)
    v, w = v[order], w[order]
    c = np.cumsum(w)
    tot = c[-1]
    return {q: float(np.interp(q*tot, c, v)) for q in qs}

# ---- similarity weight (for future: VRAM/CPU if present; now uniform) ----
def row_weight(_row):  # keep 1.0 for now; expand later when VRAM/CPU attrs are populated
    return 1.0

# ---- band computation for a subset ----
def band_for_subset(df_sub: pd.DataFrame, gpu_count: int, term_hours: int):
    if df_sub.empty:
        return None
    w = df_sub.apply(row_weight, axis=1).values
    qs = weighted_quantiles(df_sub["effective_price_usd_per_gpu_hr"].values, w, qs=(0.10,0.50,0.90))
    price_lo = qs[0.10]; price_md = qs[0.50]; price_hi = qs[0.90]
    total_lo = price_lo * gpu_count * term_hours if np.isfinite(price_lo) else np.nan
    total_md = price_md * gpu_count * term_hours if np.isfinite(price_md) else np.nan
    total_hi = price_hi * gpu_count * term_hours if np.isfinite(price_hi) else np.nan
    return dict(
        p10=round(price_lo,4) if np.isfinite(price_lo) else None,
        p50=round(price_md,4) if np.isfinite(price_md) else None,
        p90=round(price_hi,4) if np.isfinite(price_hi) else None,
        total_lo=round(total_lo,2) if np.isfinite(total_lo) else None,
        total_p50=round(total_md,2) if np.isfinite(total_md) else None,
        total_hi=round(total_hi,2) if np.isfinite(total_hi) else None,
        n=int(len(df_sub))
    )

# ---- main snapshot builder ----
def compute_calculator_snapshot(
    priced_df: pd.DataFrame,
    region: str = "US",
    term: str = "6 months",
    gpu_count: int = 8,
    allowed_types=("On-Demand",),          # only compare On-Demand by default
    include_overall=True                   # add Overall (H100+H200) row
) -> pd.DataFrame:
    # basic hygiene
    df = priced_df.copy()
    df = df[df["effective_price_usd_per_gpu_hr"].notna()]
    df["provider_type"] = df["provider"].map(infer_provider_type)

    term_hours = hours_for_term(term)

    rows = []
    for model in ["H100","H200"]:
        base = df[df["gpu_model"].eq(model)]
        if base.empty:
            continue

        # type filter
        if allowed_types:
            base = base[base["type"].isin(allowed_types)]

        # region preference: exact -> Global -> mixed
        pref = base[base["region"].eq(region)]
        used_region = region
        if pref.empty:
            pref = base[base["region"].str.lower().eq("global")]
            used_region = "Global"
        if pref.empty:
            pref = base
            used_region = "mixed"

        # one row per provider_type
        for ptype, grp in pref.groupby("provider_type", dropna=False):
            band = band_for_subset(grp, gpu_count, term_hours)
            if not band:
                continue
            rows.append({
                "gpu_model": model,
                "provider_type": ptype or "Unknown",
                "$/GPU-hr": f"${band['p10']:.2f}–${band['p90']:.2f}" if band["p10"] is not None else None,
                "TOTAL":    f"${int(band['total_lo']):,}–${int(band['total_hi']):,}" if band["total_lo"] is not None else None,
                "price_lo": band["p10"], "price_md": band["p50"], "price_hi": band["p90"],
                "term_hours": term_hours, "gpu_count": gpu_count,
                "used_region": used_region, "n_quotes": band["n"], "source": "latest"
            })

        # market average (all provider types) for this model
        band_all = band_for_subset(pref, gpu_count, term_hours)
        if band_all:
            rows.append({
                "gpu_model": model,
                "provider_type": "All",
                "$/GPU-hr": f"${band_all['p10']:.2f}–${band_all['p90']:.2f}" if band_all["p10"] is not None else None,
                "TOTAL":    f"${int(band_all['total_lo']):,}–${int(band_all['total_hi']):,}" if band_all["total_lo"] is not None else None,
                "price_lo": band_all["p10"], "price_md": band_all["p50"], "price_hi": band_all["p90"],
                "term_hours": term_hours, "gpu_count": gpu_count,
                "used_region": used_region, "n_quotes": band_all["n"], "source": "latest"
            })

    # optional overall band across both H100+H200 (uses the same regional preference)
    if include_overall:
        base = df[df["type"].isin(allowed_types)]
        pref = base[base["region"].eq(region)]
        used_region = region
        if pref.empty:
            pref = base[base["region"].str.lower().eq("global")]
            used_region = "Global"
        if pref.empty:
            pref = base
            used_region = "mixed"
        band_overall = band_for_subset(pref, gpu_count, term_hours)
        if band_overall:
            rows.append({
                "gpu_model": "H100+H200",
                "provider_type": "All",
                "$/GPU-hr": f"${band_overall['p10']:.2f}–${band_overall['p90']:.2f}" if band_overall["p10"] is not None else None,
                "TOTAL":    f"${int(band_overall['total_lo']):,}–${int(band_overall['total_hi']):,}" if band_overall["total_lo"] is not None else None,
                "price_lo": band_overall["p10"], "price_md": band_overall["p50"], "price_hi": band_overall["p90"],
                "term_hours": term_hours, "gpu_count": gpu_count,
                "used_region": used_region, "n_quotes": band_overall["n"], "source": "latest"
            })

    out = pd.DataFrame(rows).sort_values(["gpu_model","provider_type"]).reset_index(drop=True)
    return out

# ---- build and save snapshot (example: US, 6 months, 8 GPUs) ----
calc_df = compute_calculator_snapshot(
    scored,            # from your earlier pipeline
    region="US",
    term="6 months",
    gpu_count=8,
    allowed_types=("On-Demand",),   # stick to On-Demand comparisons
    include_overall=True
)
snap_path = DERIVED / "compute_calculator_snapshot.csv"
calc_df.to_csv(snap_path, index=False)
print(f"✓ wrote {snap_path} (rows: {len(calc_df)})")
display(calc_df)


✓ wrote docs/data/derived/compute_calculator_snapshot.csv (rows: 5)


,gpu_model,provider_type,$/GPU-hr,TOTAL,price_lo,price_md,price_hi,term_hours,gpu_count,used_region,n_quotes,source
0,H100,All,$1.99–$1.99,"$68,774–$68,774",1.99,1.99,1.990,4320,8,US,1,latest
1,H100,Neocloud,$1.99–$1.99,"$68,774–$68,774",1.99,1.99,1.990,4320,8,US,1,latest
2,H100+H200,All,$1.38–$2.18,"$47,692–$75,202",1.38,1.40,2.176,4320,8,US,4,latest
3,H200,All,$1.38–$2.03,"$47,692–$70,156",1.38,1.39,2.030,4320,8,US,3,latest
4,H200,Neocloud,$1.38–$2.03,"$47,692–$70,156",1.38,1.39,2.030,4320,8,US,3,latest


In [132]:
# ---------- Compute Calculator snapshot (provider buckets + robust region fallback) ----------
# Depends on: PRICE_COL, ensure_price_col(), _exclude_providers(), od (On-Demand pool)

# 1) Provider bucket classifier (keep names you actually scrape)
PROVIDER_TYPE_MAP = {
    # Neoclouds
    "Lambda Labs": "Neocloud",
    "VoltagePark": "Neocloud",
    "Paperspace":  "Neocloud",
    "Shadeform":   "Neocloud",
    "Nebius":      "Neocloud",
    "CrusoeCloud": "Neocloud",
    "SFCompute":   "Neocloud",
    "OVHcloud":    "Hyperscaler",   # use as hyperscaler stand-in until AWS/GCP/Azure scrapers exist
    # Brokers / Marketplaces
    "Vast.ai":     "Broker",
    "RunPod":      "Broker",
    "TensorDock":  "Broker",
    "Hydra Host (Brokkr)": "Broker",
}

def classify_provider_type(name: str) -> str:
    n = str(name or "").strip()
    if n in PROVIDER_TYPE_MAP:
        return PROVIDER_TYPE_MAP[n]
    # fallbacks
    if re.search(r"vast|runpod|tensor|brokkr|broker", n, flags=re.I): return "Broker"
    if re.search(r"aws|azure|gcp|google|ovh|oci|oracle", n, flags=re.I): return "Hyperscaler"
    return "Neocloud"

# 2) Prep dataset (US with Global fallback; exclude CoreWeave)
def _region_pool(df: pd.DataFrame, prefer_region="US", min_quotes=8) -> (pd.DataFrame, str):
    us  = df[df["region"] == prefer_region]
    glb = df[df["region"].str.lower().eq("global")]
    if len(us) >= min_quotes:
        return us.copy(), prefer_region
    # union (not replace) to keep any US quotes
    pool = pd.concat([us, glb], ignore_index=True) if len(glb) else us
    used = prefer_region if len(us) else ("Global" if len(glb) else "mixed")
    return pool, used

base = od.copy()
base = _exclude_providers(base)
base = ensure_price_col(base)
base["provider_type"] = base["provider"].map(classify_provider_type)

# 3) Stats per bucket
def _bucket_stats(df: pd.DataFrame, term_months=6, gpu_count=8):
    term_hours = int(term_months * 30 * 24)      # 6 months ≈ 4320h
    vals = df[PRICE_COL].astype(float).dropna().sort_values()
    n = len(vals)
    if n == 0:
        return None
    # robust quantiles (fallback for tiny samples)
    p10 = float(vals.quantile(0.10)) if n >= 5 else float(vals.min())
    p50 = float(vals.quantile(0.50)) if n >= 2 else float(vals.iloc[0])
    p90 = float(vals.quantile(0.90)) if n >= 5 else float(vals.max())
    def _fmt_range(lo, hi):
        if not np.isfinite(lo) or not np.isfinite(hi): return "—"
        if abs(lo - hi) < 1e-6: return f"${lo:.2f}"
        return f"${lo:.2f}–${hi:.2f}"
    price_str = _fmt_range(p10, p90)
    total_lo  = p10 * gpu_count * term_hours
    total_hi  = p90 * gpu_count * term_hours
    total_str = _fmt_range(round(total_lo), round(total_hi)).replace(".00", "")
    return {
        "$/GPU-hr": price_str,
        "TOTAL": total_str,
        "price_lo": round(p10, 2),
        "price_md": round(p50, 2),
        "price_hi": round(p90, 2),
        "term_hours": term_hours,
        "gpu_count": int(gpu_count),
        "n_quotes": int(n),
        "source": "latest",
    }

def make_snapshot_rows(pool_df: pd.DataFrame, used_region: str, gpu_model_label: str):
    rows = []
    # ALL
    stats_all = _bucket_stats(pool_df)
    if stats_all:
        rows.append({"gpu_model": gpu_model_label, "provider_type": "All",
                     "used_region": used_region, **stats_all})
    # Neocloud
    st = _bucket_stats(pool_df[pool_df["provider_type"]=="Neocloud"])
    if st:
        rows.append({"gpu_model": gpu_model_label, "provider_type": "Neocloud",
                     "used_region": used_region, **st})
    # Broker
    st = _bucket_stats(pool_df[pool_df["provider_type"]=="Broker"])
    if st:
        rows.append({"gpu_model": gpu_model_label, "provider_type": "Broker",
                     "used_region": used_region, **st})
    # Hyperscaler (will be filled by OVHcloud until AWS/GCP/Azure scrapers are added)
    st = _bucket_stats(pool_df[pool_df["provider_type"]=="Hyperscaler"])
    if st:
        rows.append({"gpu_model": gpu_model_label, "provider_type": "Hyperscaler",
                     "used_region": used_region, **st})
    return rows

snap_rows = []

# H100
h100 = base[base["gpu_model"]=="H100"]
h100_pool, reg_h100 = _region_pool(h100, prefer_region="US", min_quotes=8)
snap_rows += make_snapshot_rows(h100_pool, reg_h100, "H100")

# H200
h200 = base[base["gpu_model"]=="H200"]
h200_pool, reg_h200 = _region_pool(h200, prefer_region="US", min_quotes=8)
snap_rows += make_snapshot_rows(h200_pool, reg_h200, "H200")

# H100+H200 combined (All only)
both_pool, reg_both = _region_pool(base, prefer_region="US", min_quotes=12)
st = _bucket_stats(both_pool)
if st:
    snap_rows.append({"gpu_model": "H100+H200", "provider_type": "All",
                      "used_region": reg_both, **st})

snap_df = (pd.DataFrame(snap_rows)
             .loc[:, ["gpu_model","provider_type","$/GPU-hr","TOTAL",
                      "price_lo","price_md","price_hi",
                      "term_hours","gpu_count","used_region","n_quotes","source"]])

snap_path = DERIVED_DIR / "compute_calculator_snapshot.csv"
snap_df.to_csv(snap_path, index=False)
print(f"✓ wrote {snap_path} (rows: {len(snap_df)})")
display(snap_df)


✓ wrote docs/data/derived/compute_calculator_snapshot.csv (rows: 8)


,gpu_model,provider_type,$/GPU-hr,TOTAL,price_lo,price_md,price_hi,term_hours,gpu_count,used_region,n_quotes,source
0,H100,All,$1.99–$2.99,$68774–$103421,1.99,2.25,2.99,4320,8,US,11,latest
1,H100,Neocloud,$1.99–$3.12,$68774–$108000,1.99,2.12,3.12,4320,8,US,6,latest
2,H100,Broker,$1.25–$2.30,$43200–$79488,1.25,2.25,2.30,4320,8,US,3,latest
3,H100,Hyperscaler,$2.99–$2.99,$103334–$103421,2.99,2.99,2.99,4320,8,US,2,latest
4,H200,All,$1.39–$3.22,$48108–$111145,1.39,2.30,3.22,4320,8,US,7,latest
5,H200,Neocloud,$1.39–$3.37,$48038–$116467,1.39,2.30,3.37,4320,8,US,6,latest
6,H200,Broker,$2.50,$86400,2.50,2.50,2.50,4320,8,US,1,latest
7,H100+H200,All,$1.39–$3.26,$48177–$112830,1.39,2.30,3.26,4320,8,US,18,latest
